## notebook to subset and recalculate for different regions

in prep for creating histograms

In [1]:
# import my modules
import sys
sys.path.append('../')  # import functions to make life easier
import climate_xr_funcs
import climate_droughts_xr_funcs

# import things
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cftime
import regionmask
# import salem
import geopandas as gp
import pandas as pd
import requests
import io
from dask.diagnostics import ProgressBar

# this is all for plotting...
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.colors import BoundaryNorm
from matplotlib.colors import LinearSegmentedColormap
import cartopy.feature as cfeature
import matplotlib.mlab as mlab
import cartopy.io.shapereader as shapereader
from dask.diagnostics import ProgressBar

import os

# from matplotlib.offsetbox import AnchoredText

from scipy import stats

import matplotlib as mpl

Definitions

In [2]:
def droughts_lastmillennium_fromyear(ds, threshold_startyear, threshold_endyear):
#     print(ds)
    ds2 = ds.PRECT_mm
    ds3 = ds2.to_dataset()
#     print(ds2)
    ff_precip_annual = ds3
    ff_precip_annual_threshold = ff_precip_annual.where((ff_precip_annual['year'] >= threshold_startyear) 
                                                        & (ff_precip_annual['year'] <= threshold_endyear), drop=True)
    # get years that are drought vs not drought
    ff_precip_annual['drought_years_2s2e'] = climate_droughts_xr_funcs.get_drought_years_2S2E(ff_precip_annual.PRECT_mm, ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['drought_years_median'] = ('year', climate_droughts_xr_funcs.get_drought_years_below_threshold(ff_precip_annual.PRECT_mm, ff_precip_annual_threshold.PRECT_mm.quantile(0.5, dim='year')))
    ff_precip_annual['drought_years_20perc'] = ('year', climate_droughts_xr_funcs.get_drought_years_below_threshold(ff_precip_annual.PRECT_mm, ff_precip_annual_threshold.PRECT_mm.quantile(0.2, dim='year')))   
    ff_precip_annual['drought_years_120pc_2med'] = (climate_droughts_xr_funcs.get_drought_years_120perc_2median(ff_precip_annual.PRECT_mm, ff_precip_annual_threshold.PRECT_mm.quantile(0.2, dim='year'), ff_precip_annual_threshold.PRECT_mm.quantile(0.5, dim='year')))
    ff_precip_annual['drought_years_220pc_1med'] = (climate_droughts_xr_funcs.get_drought_years_below_start_end_thresholds(ff_precip_annual.PRECT_mm, ff_precip_annual_threshold.PRECT_mm.quantile(0.2, dim='year'), ff_precip_annual_threshold.PRECT_mm.quantile(0.5, dim='year')))

    # get overall length of droughts
    ff_precip_annual['droughts_2s2e'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_2s2e'])
    ff_precip_annual['droughts_median'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_median'])
    ff_precip_annual['droughts_20perc'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_20perc'])
    ff_precip_annual['droughts_120pc_2med'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_120pc_2med'])
    ff_precip_annual['droughts_220pc_1med'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_220pc_1med'])
    
    # get max length in this period
    ff_precip_annual['droughts_2s2e_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_2s2e, dim='year')
    ff_precip_annual['droughts_median_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_median, dim='year')
    ff_precip_annual['droughts_20perc_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_20perc, dim='year')
    ff_precip_annual['droughts_120pc_2med_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_120pc_2med, dim='year')
    ff_precip_annual['droughts_220pc_1med_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_220pc_1med, dim='year')
    
    # get mean length in this period
    ff_precip_annual['droughts_2s2e_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_2s2e, dim='year')
    ff_precip_annual['droughts_median_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_median, dim='year')
    ff_precip_annual['droughts_20perc_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_20perc, dim='year')
    ff_precip_annual['droughts_120pc_2med_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_120pc_2med, dim='year')
    ff_precip_annual['droughts_220pc_1med_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_220pc_1med, dim='year')
    
    # count how many individual events occur
    ff_precip_annual['droughts_2s2e_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_2s2e)
    ff_precip_annual['droughts_median_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_median)
    ff_precip_annual['droughts_20perc_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_20perc)
    ff_precip_annual['droughts_120pc_2med_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_120pc_2med)
    ff_precip_annual['droughts_220pc_1med_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_220pc_1med)
    
    ff_precip_annual['droughts_2s2e_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_2s2e, dim='year')
    ff_precip_annual['droughts_median_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_median, dim='year')
    ff_precip_annual['droughts_20perc_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_20perc, dim='year')
    ff_precip_annual['droughts_120pc_2med_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_120pc_2med, dim='year')
    ff_precip_annual['droughts_220pc_1med_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_220pc_1med, dim='year')
    
    ff_precip_annual['droughts_2s2e_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_2s2e', 'droughts_2s2e', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_median_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_median', 'droughts_median', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_20perc_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_20perc', 'droughts_20perc', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_120pc_2med_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_120pc_2med', 'droughts_120pc_2med', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_220pc_1med_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_220pc_1med', 'droughts_220pc_1med', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))

    ff_precip_annual['droughts_2s2e_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_2s2e', 'droughts_2s2e', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_median_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_median', 'droughts_median', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_20perc_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_20perc', 'droughts_20perc', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_120pc_2med_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_120pc_2med', 'droughts_120pc_2med', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_220pc_1med_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_220pc_1med', 'droughts_220pc_1med', ff_precip_annual_threshold.PRECT_mm.mean(dim='year'))
    
    return ff_precip_annual

In [3]:
def save_netcdf_compression(ds, output_dir, filename):

    comp = dict(zlib=True, complevel=8)
    encoding = {var: comp for var in ds.data_vars}

    delayed_obj = ds.to_netcdf('%s/%s.nc' % (output_dir, filename), mode='w', compute=False, encoding=encoding)
    with ProgressBar():
        results = delayed_obj.compute()       

In [4]:
def get_aus(ds):
    mask = regionmask.defined_regions.natural_earth.countries_110.mask(ds)
    ds_aus = ds.where(mask == 137, drop=True)
    return ds_aus

In [5]:
def droughts_bootstrap(ds, model_name):
    
    ds['PRECT_mm'] = ds[model_name]
    ds2 = ds.PRECT_mm
    ds3 = ds2.to_dataset()
#     print(ds2)
    ff_precip_annual = ds3

    # get years that are drought vs not drought
    ff_precip_annual['drought_years_2s2e'] = climate_droughts_xr_funcs.get_drought_years_2S2E(ff_precip_annual.PRECT_mm, ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['drought_years_median'] = ('year', climate_droughts_xr_funcs.get_drought_years_below_threshold(ff_precip_annual.PRECT_mm, ff_precip_annual.PRECT_mm.quantile(0.5, dim='year')))
    ff_precip_annual['drought_years_20perc'] = ('year', climate_droughts_xr_funcs.get_drought_years_below_threshold(ff_precip_annual.PRECT_mm, ff_precip_annual.PRECT_mm.quantile(0.2, dim='year')))
    ff_precip_annual['drought_years_120pc_2med'] = ('year', climate_droughts_xr_funcs.get_drought_years_120perc_2median(ff_precip_annual.PRECT_mm, ff_precip_annual.PRECT_mm.quantile(0.2, dim='year'), ff_precip_annual.PRECT_mm.quantile(0.5, dim='year')))
    ff_precip_annual['drought_years_220pc_1med'] = ('year', climate_droughts_xr_funcs.get_drought_years_below_start_end_thresholds(ff_precip_annual.PRECT_mm, ff_precip_annual.PRECT_mm.quantile(0.2, dim='year'), ff_precip_annual.PRECT_mm.quantile(0.5, dim='year')))

    # get overall length of droughts
    ff_precip_annual['droughts_2s2e'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_2s2e'])
    ff_precip_annual['droughts_median'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_median'])
    ff_precip_annual['droughts_20perc'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_20perc'])
    ff_precip_annual['droughts_120pc_2med'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_120pc_2med'])
    ff_precip_annual['droughts_220pc_1med'] = climate_droughts_xr_funcs.cumulative_drought_length(ff_precip_annual['drought_years_220pc_1med'])
    
    # get max length in this period
    ff_precip_annual['droughts_2s2e_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_2s2e, dim='year')
    ff_precip_annual['droughts_median_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_median, dim='year')
    ff_precip_annual['droughts_20perc_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_20perc, dim='year')
    ff_precip_annual['droughts_120pc_2med_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_120pc_2med, dim='year')
    ff_precip_annual['droughts_220pc_1med_max'] = climate_droughts_xr_funcs.max_length_ufunc(ff_precip_annual.droughts_220pc_1med, dim='year')
    
    # get mean length in this period
    ff_precip_annual['droughts_2s2e_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_2s2e, dim='year')
    ff_precip_annual['droughts_median_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_median, dim='year')
    ff_precip_annual['droughts_20perc_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_20perc, dim='year')
    ff_precip_annual['droughts_120pc_2med_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_120pc_2med, dim='year')
    ff_precip_annual['droughts_220pc_1med_mean'] = climate_droughts_xr_funcs.mean_length_ufunc(ff_precip_annual.droughts_220pc_1med, dim='year')
    
    # count how many individual events occur
    ff_precip_annual['droughts_2s2e_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_2s2e)
    ff_precip_annual['droughts_median_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_median)
    ff_precip_annual['droughts_20perc_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_20perc)
    ff_precip_annual['droughts_120pc_2med_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_120pc_2med)
    ff_precip_annual['droughts_220pc_1med_no_of_events'] = climate_droughts_xr_funcs.count_drought_events(ff_precip_annual.droughts_220pc_1med)
    
    ff_precip_annual['droughts_2s2e_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_2s2e, dim='year')
    ff_precip_annual['droughts_median_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_median, dim='year')
    ff_precip_annual['droughts_20perc_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_20perc, dim='year')
    ff_precip_annual['droughts_120pc_2med_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_120pc_2med, dim='year')
    ff_precip_annual['droughts_220pc_1med_std'] = climate_droughts_xr_funcs.std_apply(ff_precip_annual.droughts_220pc_1med, dim='year')
    
    ff_precip_annual['droughts_2s2e_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_2s2e', 'droughts_2s2e', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_median_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_median', 'droughts_median', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_20perc_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_20perc', 'droughts_20perc', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_120pc_2med_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_120pc_2med', 'droughts_120pc_2med', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_220pc_1med_intensity'] = climate_droughts_xr_funcs.drought_intensity(ff_precip_annual, 'drought_years_220pc_1med', 'droughts_220pc_1med', ff_precip_annual.PRECT_mm.mean(dim='year'))

    ff_precip_annual['droughts_2s2e_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_2s2e', 'droughts_2s2e', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_median_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_median', 'droughts_median', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_20perc_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_20perc', 'droughts_20perc', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_120pc_2med_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_120pc_2med', 'droughts_120pc_2med', ff_precip_annual.PRECT_mm.mean(dim='year'))
    ff_precip_annual['droughts_220pc_1med_severity'] = climate_droughts_xr_funcs.drought_severity(ff_precip_annual, 'drought_years_220pc_1med', 'droughts_220pc_1med', ff_precip_annual.PRECT_mm.mean(dim='year'))
    
    return ff_precip_annual



def bootstrap_years_101(ds, model_name):
    ds_boots = []    
    years = ds.year.values
    
    for i in years:
        if i > 1900:
            pass
        else:
            ds_sub = ds.sel(year=slice(i, i+100))
            ds_sub2 = ds_sub.to_dataset()
            ds_out = droughts_bootstrap(ds_sub2, model_name)
            ds_boots.append(ds_out)
    ds_boots_out = xr.concat(ds_boots, dim='n')
        
    return ds_boots_out
    

## Import previously made Aus files

In [9]:
historical_year = 1900
input_dir_hist = '../files/historical_%s/aus' % historical_year

ff1_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '1'))
ff2_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '2'))
ff3_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '3'))
ff4_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '4'))
ff5_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '5'))
ff6_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '6'))
ff7_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '7'))
ff8_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '8'))
ff9_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '9'))
ff10_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '10'))
ff11_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '11'))
ff12_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '12'))
ff13_precip_hist_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_hist_annual_aus.nc' % (input_dir_hist, '13'))

bcc_precip_hist_annual_aus = xr.open_dataset('%s/bcc_precip_hist_annual_aus.nc' % input_dir_hist)
ccsm4_precip_hist_annual_aus = xr.open_dataset('%s/ccsm4_precip_hist_annual_aus.nc' % input_dir_hist)
csiro_mk3l_precip_hist_annual_aus = xr.open_dataset('%s/csiro_mk3l_precip_hist_annual_aus.nc' % input_dir_hist)
fgoals_gl_precip_hist_annual_aus = xr.open_dataset('%s/fgoals_gl_precip_hist_annual_aus.nc' % input_dir_hist)
fgoals_s2_precip_hist_annual_aus = xr.open_dataset('%s/fgoals_s2_precip_hist_annual_aus.nc' % input_dir_hist)
giss_21_precip_hist_annual_aus = xr.open_dataset('%s/giss_21_precip_hist_annual_aus.nc' % input_dir_hist)
giss_22_precip_hist_annual_aus = xr.open_dataset('%s/giss_22_precip_hist_annual_aus.nc' % input_dir_hist)
giss_23_precip_hist_annual_aus = xr.open_dataset('%s/giss_23_precip_hist_annual_aus.nc' % input_dir_hist)
giss_24_precip_hist_annual_aus = xr.open_dataset('%s/giss_24_precip_hist_annual_aus.nc' % input_dir_hist)
giss_25_precip_hist_annual_aus = xr.open_dataset('%s/giss_25_precip_hist_annual_aus.nc' % input_dir_hist)
giss_26_precip_hist_annual_aus = xr.open_dataset('%s/giss_26_precip_hist_annual_aus.nc' % input_dir_hist)
giss_27_precip_hist_annual_aus = xr.open_dataset('%s/giss_27_precip_hist_annual_aus.nc' % input_dir_hist)
giss_28_precip_hist_annual_aus = xr.open_dataset('%s/giss_28_precip_hist_annual_aus.nc' % input_dir_hist)
hadcm3_precip_hist_annual_aus = xr.open_dataset('%s/hadcm3_precip_hist_annual_aus.nc' % input_dir_hist)
ipsl_precip_hist_annual_aus = xr.open_dataset('%s/ipsl_precip_hist_annual_aus.nc' % input_dir_hist)
miroc_precip_hist_annual_aus = xr.open_dataset('%s/miroc_precip_hist_annual_aus.nc' % input_dir_hist)
mpi_precip_hist_annual_aus = xr.open_dataset('%s/mpi_precip_hist_annual_aus.nc' % input_dir_hist)
mri_precip_hist_annual_aus = xr.open_dataset('%s/mri_precip_hist_annual_aus.nc' % input_dir_hist)

In [10]:
input_dir_lm = '../files/lastmillennium_threshold_1900-2000/aus'

ff1_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '1'))
ff2_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '2'))
ff3_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '3'))
ff4_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '4'))
ff5_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '5'))
ff6_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '6'))
ff7_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '7'))
ff8_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '8'))
ff9_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '9'))
ff10_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '10'))
ff11_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '11'))
ff12_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '12'))
ff13_precip_lm_annual_aus = xr.open_dataset('%s/cesmlme-ff%s_precip_lm_annual_aus.nc' % (input_dir_lm, '13'))

bcc_precip_lm_annual_aus = xr.open_dataset('%s/bcc_precip_lm_annual_aus.nc' % input_dir_lm)
ccsm4_precip_lm_annual_aus = xr.open_dataset('%s/ccsm4_precip_lm_annual_aus.nc' % input_dir_lm)
csiro_mk3l_precip_lm_annual_aus = xr.open_dataset('%s/csiro_mk3l_precip_lm_annual_aus.nc' % input_dir_lm)
fgoals_gl_precip_lm_annual_aus = xr.open_dataset('%s/fgoals_gl_precip_lm_annual_aus.nc' % input_dir_lm)
fgoals_s2_precip_lm_annual_aus = xr.open_dataset('%s/fgoals_s2_precip_lm_annual_aus.nc' % input_dir_lm)
giss_21_precip_lm_annual_aus = xr.open_dataset('%s/giss_21_precip_lm_annual_aus.nc' % input_dir_lm)
giss_22_precip_lm_annual_aus = xr.open_dataset('%s/giss_22_precip_lm_annual_aus.nc' % input_dir_lm)
giss_23_precip_lm_annual_aus = xr.open_dataset('%s/giss_23_precip_lm_annual_aus.nc' % input_dir_lm)
giss_24_precip_lm_annual_aus = xr.open_dataset('%s/giss_24_precip_lm_annual_aus.nc' % input_dir_lm)
giss_25_precip_lm_annual_aus = xr.open_dataset('%s/giss_25_precip_lm_annual_aus.nc' % input_dir_lm)
giss_26_precip_lm_annual_aus = xr.open_dataset('%s/giss_26_precip_lm_annual_aus.nc' % input_dir_lm)
giss_27_precip_lm_annual_aus = xr.open_dataset('%s/giss_27_precip_lm_annual_aus.nc' % input_dir_lm)
giss_28_precip_lm_annual_aus = xr.open_dataset('%s/giss_28_precip_lm_annual_aus.nc' % input_dir_lm)
hadcm3_precip_lm_annual_aus = xr.open_dataset('%s/hadcm3_precip_lm_annual_aus.nc' % input_dir_lm)
ipsl_precip_lm_annual_aus = xr.open_dataset('%s/ipsl_precip_lm_annual_aus.nc' % input_dir_lm)
miroc_precip_lm_annual_aus = xr.open_dataset('%s/miroc_precip_lm_annual_aus.nc' % input_dir_lm)
mpi_precip_lm_annual_aus = xr.open_dataset('%s/mpi_precip_lm_annual_aus.nc' % input_dir_lm)
mri_precip_lm_annual_aus = xr.open_dataset('%s/mri_precip_lm_annual_aus.nc' % input_dir_lm)

control

In [8]:
input_dir_ctrl = '../files/control'

In [9]:
bcc_annual_cntl_aus = xr.open_dataset('%s/bcc_precip_cntl_annual_aus.nc' % input_dir_ctrl)
ccsm4_annual_cntl_aus = xr.open_dataset('%s/ccsm4_precip_cntl_annual_aus.nc' % input_dir_ctrl)
csiro_mk3l_annual_cntl_aus = xr.open_dataset('%s/csiro_mk3l_precip_cntl_annual_aus.nc' % input_dir_ctrl)
fgoals_s2_annual_cntl_aus = xr.open_dataset('%s/fgoals_s2_precip_cntl_annual_aus.nc' % input_dir_ctrl)
giss_2_annual_cntl_aus = xr.open_dataset('%s/giss_2_precip_cntl_annual_aus.nc' % input_dir_ctrl)
hadcm3_annual_cntl_aus = xr.open_dataset('%s/hadcm3_precip_cntl_annual_aus.nc' % input_dir_ctrl)
ipsl_annual_cntl_aus = xr.open_dataset('%s/ipsl_precip_cntl_annual_aus.nc' % input_dir_ctrl)
miroc_annual_cntl_aus = xr.open_dataset('%s/miroc_precip_cntl_annual_aus.nc' % input_dir_ctrl)
mpi_annual_cntl_aus = xr.open_dataset('%s/mpi_precip_cntl_annual_aus.nc' % input_dir_ctrl)
mri_annual_cntl_aus = xr.open_dataset('%s/mri_precip_cntl_annual_aus.nc' % input_dir_ctrl)
giss_3_annual_cntl_aus = xr.open_dataset('%s/giss_3_precip_cntl_annual_aus.nc' % input_dir_ctrl)
giss_1_annual_cntl_aus = xr.open_dataset('%s/giss_1_precip_cntl_annual_aus.nc' % input_dir_ctrl)
giss_41_annual_cntl_aus = xr.open_dataset('%s/giss_41_precip_cntl_annual_aus.nc' % input_dir_ctrl)

### import BoM

In [16]:
# ## import bom files
# bom_annual_precip_eastern_url= 'http://www.bom.gov.au/web01/ncc/www/cli_chg/timeseries/rain/0112/eaus/latest.txt'
# s=requests.get(bom_annual_precip_eastern_url).content
# bom_eastern_precip = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='  ',names=['time', 'precip'])
# print(bom_eastern_precip)
# bom_eastern_precip['year'] = bom_eastern_precip.time.astype(str).str[:4].astype(int)

# bom_southeastern_precip_url= 'http://www.bom.gov.au/web01/ncc/www/cli_chg/timeseries/rain/0112/seaus/latest.txt'
# s=requests.get(bom_southeastern_precip_url).content
# bom_southeastern_precip = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='  ',names=['time', 'precip'])
# bom_southeastern_precip['year'] = bom_southeastern_precip.time.astype(str).str[:4].astype(int)

# bom_southern_precip_url= 'http://www.bom.gov.au/web01/ncc/www/cli_chg/timeseries/rain/0112/saus/latest.txt'
# s=requests.get(bom_southern_precip_url).content
# bom_southern_precip = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='  ',names=['time', 'precip'])
# bom_southern_precip['year'] = bom_southern_precip.time.astype(str).str[:4].astype(int)

# bom_northern_precip_url= 'http://www.bom.gov.au/web01/ncc/www/cli_chg/timeseries/rain/0112/naus/latest.txt'
# s=requests.get(bom_northern_precip_url).content
# bom_northern_precip = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='  ',names=['time', 'precip'])
# bom_northern_precip['year'] = bom_northern_precip.time.astype(str).str[:4].astype(int)

## Limit historical to 2000 (number of years: 101)

In [17]:
ff1_precip_hist_annual_aus = ff1_precip_hist_annual_aus.where(ff1_precip_hist_annual_aus.year <= 2000, drop=True)
ff2_precip_hist_annual_aus = ff2_precip_hist_annual_aus.where(ff2_precip_hist_annual_aus.year <= 2000, drop=True)
ff3_precip_hist_annual_aus = ff3_precip_hist_annual_aus.where(ff3_precip_hist_annual_aus.year <= 2000, drop=True)
ff4_precip_hist_annual_aus = ff4_precip_hist_annual_aus.where(ff4_precip_hist_annual_aus.year <= 2000, drop=True)
ff5_precip_hist_annual_aus = ff5_precip_hist_annual_aus.where(ff5_precip_hist_annual_aus.year <= 2000, drop=True)
ff6_precip_hist_annual_aus = ff6_precip_hist_annual_aus.where(ff6_precip_hist_annual_aus.year <= 2000, drop=True)
ff7_precip_hist_annual_aus = ff7_precip_hist_annual_aus.where(ff7_precip_hist_annual_aus.year <= 2000, drop=True)
ff8_precip_hist_annual_aus = ff8_precip_hist_annual_aus.where(ff8_precip_hist_annual_aus.year <= 2000, drop=True)
ff9_precip_hist_annual_aus = ff9_precip_hist_annual_aus.where(ff9_precip_hist_annual_aus.year <= 2000, drop=True)
ff10_precip_hist_annual_aus = ff10_precip_hist_annual_aus.where(ff10_precip_hist_annual_aus.year <= 2000, drop=True)
ff11_precip_hist_annual_aus = ff11_precip_hist_annual_aus.where(ff11_precip_hist_annual_aus.year <= 2000, drop=True)
ff12_precip_hist_annual_aus = ff12_precip_hist_annual_aus.where(ff12_precip_hist_annual_aus.year <= 2000, drop=True)
ff13_precip_hist_annual_aus = ff13_precip_hist_annual_aus.where(ff13_precip_hist_annual_aus.year <= 2000, drop=True)

bcc_precip_hist_annual_aus        = bcc_precip_hist_annual_aus.where(bcc_precip_hist_annual_aus.year <=2000, drop=True)
ccsm4_precip_hist_annual_aus      = ccsm4_precip_hist_annual_aus.where(ccsm4_precip_hist_annual_aus.year <=2000, drop=True)
csiro_mk3l_precip_hist_annual_aus = csiro_mk3l_precip_hist_annual_aus.where(csiro_mk3l_precip_hist_annual_aus.year <=2000, drop=True)
fgoals_gl_precip_hist_annual_aus  = fgoals_gl_precip_hist_annual_aus.where(fgoals_gl_precip_hist_annual_aus.year <=2000, drop=True)
fgoals_s2_precip_hist_annual_aus  = fgoals_s2_precip_hist_annual_aus.where(fgoals_s2_precip_hist_annual_aus.year <=2000, drop=True)
giss_21_precip_hist_annual_aus    = giss_21_precip_hist_annual_aus.where(giss_21_precip_hist_annual_aus.year <=2000, drop=True)
giss_22_precip_hist_annual_aus    = giss_22_precip_hist_annual_aus.where(giss_22_precip_hist_annual_aus.year <=2000, drop=True)
giss_23_precip_hist_annual_aus    = giss_23_precip_hist_annual_aus.where(giss_23_precip_hist_annual_aus.year <=2000, drop=True)
giss_24_precip_hist_annual_aus    = giss_24_precip_hist_annual_aus.where(giss_24_precip_hist_annual_aus.year <=2000, drop=True)
giss_25_precip_hist_annual_aus    = giss_25_precip_hist_annual_aus.where(giss_25_precip_hist_annual_aus.year <=2000, drop=True)
giss_26_precip_hist_annual_aus    = giss_26_precip_hist_annual_aus.where(giss_26_precip_hist_annual_aus.year <=2000, drop=True)
giss_27_precip_hist_annual_aus    = giss_27_precip_hist_annual_aus.where(giss_27_precip_hist_annual_aus.year <=2000, drop=True)
giss_28_precip_hist_annual_aus    = giss_28_precip_hist_annual_aus.where(giss_28_precip_hist_annual_aus.year <=2000, drop=True)
hadcm3_precip_hist_annual_aus     = hadcm3_precip_hist_annual_aus.where(hadcm3_precip_hist_annual_aus.year <=2000, drop=True)
ipsl_precip_hist_annual_aus       = ipsl_precip_hist_annual_aus.where(ipsl_precip_hist_annual_aus.year <=2000, drop=True)
miroc_precip_hist_annual_aus      = miroc_precip_hist_annual_aus.where(miroc_precip_hist_annual_aus.year <=2000, drop=True)
mpi_precip_hist_annual_aus      = mpi_precip_hist_annual_aus.where(mpi_precip_hist_annual_aus.year <=2000, drop=True)
mri_precip_hist_annual_aus      = mri_precip_hist_annual_aus.where(mri_precip_hist_annual_aus.year <=2000, drop=True)

In [18]:
awap_gf_annual = awap_gf_annual.where(awap_gf_annual.year <= 2000, drop=True)

---

# Recalculate drought metrics from area-averaged precip

## Murray darling basin

In [6]:
def get_mdb_mean(ds):
    # import shapefile using geopandas
    states = gp.read_file('/Users/nickywright/Data/Coastlines/Murray.shp')
    mask = regionmask.mask_geopandas(states, ds.lon, ds.lat)
    # mask out 
    ds_mdb = ds.where(mask == mask, drop=True)
    ds_mdb_mean = climate_xr_funcs.weighted_mean_CAM(ds_mdb)
    
    return ds_mdb_mean

### historical

In [20]:
ff1_precip_hist_annual_mdb       = get_mdb_mean(ff1_precip_hist_annual_aus)
ff2_precip_hist_annual_mdb       = get_mdb_mean(ff2_precip_hist_annual_aus)
ff3_precip_hist_annual_mdb       = get_mdb_mean(ff3_precip_hist_annual_aus)
ff4_precip_hist_annual_mdb       = get_mdb_mean(ff4_precip_hist_annual_aus)
ff5_precip_hist_annual_mdb       = get_mdb_mean(ff5_precip_hist_annual_aus)
ff6_precip_hist_annual_mdb       = get_mdb_mean(ff6_precip_hist_annual_aus)
ff7_precip_hist_annual_mdb       = get_mdb_mean(ff7_precip_hist_annual_aus)
ff8_precip_hist_annual_mdb       = get_mdb_mean(ff8_precip_hist_annual_aus)
ff9_precip_hist_annual_mdb       = get_mdb_mean(ff9_precip_hist_annual_aus)
ff10_precip_hist_annual_mdb      = get_mdb_mean(ff10_precip_hist_annual_aus)
ff11_precip_hist_annual_mdb      = get_mdb_mean(ff11_precip_hist_annual_aus)
ff12_precip_hist_annual_mdb      = get_mdb_mean(ff12_precip_hist_annual_aus)
ff13_precip_hist_annual_mdb      = get_mdb_mean(ff13_precip_hist_annual_aus)

bcc_precip_hist_annual_mdb        = get_mdb_mean(bcc_precip_hist_annual_aus)
ccsm4_precip_hist_annual_mdb      = get_mdb_mean(ccsm4_precip_hist_annual_aus)
csiro_mk3l_precip_hist_annual_mdb = get_mdb_mean(csiro_mk3l_precip_hist_annual_aus)
fgoals_gl_precip_hist_annual_mdb  = get_mdb_mean(fgoals_gl_precip_hist_annual_aus)
fgoals_s2_precip_hist_annual_mdb  = get_mdb_mean(fgoals_s2_precip_hist_annual_aus)
giss_21_precip_hist_annual_mdb    = get_mdb_mean(giss_21_precip_hist_annual_aus)
giss_22_precip_hist_annual_mdb    = get_mdb_mean(giss_22_precip_hist_annual_aus)
giss_23_precip_hist_annual_mdb    = get_mdb_mean(giss_23_precip_hist_annual_aus)
giss_24_precip_hist_annual_mdb    = get_mdb_mean(giss_24_precip_hist_annual_aus)
giss_25_precip_hist_annual_mdb    = get_mdb_mean(giss_25_precip_hist_annual_aus)
giss_26_precip_hist_annual_mdb    = get_mdb_mean(giss_26_precip_hist_annual_aus)
giss_27_precip_hist_annual_mdb    = get_mdb_mean(giss_27_precip_hist_annual_aus)
giss_28_precip_hist_annual_mdb    = get_mdb_mean(giss_28_precip_hist_annual_aus)
hadcm3_precip_hist_annual_mdb     = get_mdb_mean(hadcm3_precip_hist_annual_aus)
ipsl_precip_hist_annual_mdb       = get_mdb_mean(ipsl_precip_hist_annual_aus)
miroc_precip_hist_annual_mdb      = get_mdb_mean(miroc_precip_hist_annual_aus)
mpi_precip_hist_annual_mdb        = get_mdb_mean(mpi_precip_hist_annual_aus)
mri_precip_hist_annual_mdb        = get_mdb_mean(mri_precip_hist_annual_aus)

awap_masked_annual_mdb = get_mdb_mean(awap_gf_annual)

In [47]:
ff1_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff1_precip_hist_annual_mdb, 1900, 2000)    
ff2_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff2_precip_hist_annual_mdb, 1900, 2000)    
ff3_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff3_precip_hist_annual_mdb, 1900, 2000)    
ff4_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff4_precip_hist_annual_mdb, 1900, 2000)    
ff5_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff5_precip_hist_annual_mdb, 1900, 2000)    
ff6_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff6_precip_hist_annual_mdb, 1900, 2000)    
ff7_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff7_precip_hist_annual_mdb, 1900, 2000)    
ff8_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff8_precip_hist_annual_mdb, 1900, 2000)    
ff9_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff9_precip_hist_annual_mdb, 1900, 2000)    
ff10_precip_hist_annual_mdb_rc      = droughts_lastmillennium_fromyear(ff10_precip_hist_annual_mdb, 1900, 2000)    
ff11_precip_hist_annual_mdb_rc      = droughts_lastmillennium_fromyear(ff11_precip_hist_annual_mdb, 1900, 2000)    
ff12_precip_hist_annual_mdb_rc      = droughts_lastmillennium_fromyear(ff12_precip_hist_annual_mdb, 1900, 2000)    
ff13_precip_hist_annual_mdb_rc      = droughts_lastmillennium_fromyear(ff13_precip_hist_annual_mdb, 1900, 2000)    
bcc_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(bcc_precip_hist_annual_mdb, 1900, 2000)    
ccsm4_precip_hist_annual_mdb_rc     = droughts_lastmillennium_fromyear(ccsm4_precip_hist_annual_mdb     , 1900, 2000)
csiro_mk3l_precip_hist_annual_mdb_rc= droughts_lastmillennium_fromyear(csiro_mk3l_precip_hist_annual_mdb, 1900, 2000)
fgoals_gl_precip_hist_annual_mdb_rc = droughts_lastmillennium_fromyear(fgoals_gl_precip_hist_annual_mdb , 1900, 2000)
fgoals_s2_precip_hist_annual_mdb_rc = droughts_lastmillennium_fromyear(fgoals_s2_precip_hist_annual_mdb , 1900, 2000)
giss_21_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_21_precip_hist_annual_mdb   , 1900, 2000)
giss_22_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_22_precip_hist_annual_mdb   , 1900, 2000)
giss_23_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_23_precip_hist_annual_mdb   , 1900, 2000)
giss_24_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_24_precip_hist_annual_mdb   , 1900, 2000)
giss_25_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_25_precip_hist_annual_mdb   , 1900, 2000)
giss_26_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_26_precip_hist_annual_mdb   , 1900, 2000)
giss_27_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_27_precip_hist_annual_mdb   , 1900, 2000)
giss_28_precip_hist_annual_mdb_rc   = droughts_lastmillennium_fromyear(giss_28_precip_hist_annual_mdb   , 1900, 2000)
hadcm3_precip_hist_annual_mdb_rc    = droughts_lastmillennium_fromyear(hadcm3_precip_hist_annual_mdb    , 1900, 2000)
ipsl_precip_hist_annual_mdb_rc      = droughts_lastmillennium_fromyear(ipsl_precip_hist_annual_mdb      , 1900, 2000)
miroc_precip_hist_annual_mdb_rc     = droughts_lastmillennium_fromyear(miroc_precip_hist_annual_mdb     , 1900, 2000)
mpi_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(mpi_precip_hist_annual_mdb       , 1900, 2000)
mri_precip_hist_annual_mdb_rc       = droughts_lastmillennium_fromyear(mri_precip_hist_annual_mdb       , 1900, 2000)

awap_masked_annual_mdb_rc = droughts_lastmillennium_fromyear(awap_masked_annual_mdb, 1900, 2000)

ff_all_precip_hist_annual_mdb = xr.concat([ff1_precip_hist_annual_mdb, ff2_precip_hist_annual_mdb,
                                           ff3_precip_hist_annual_mdb, ff4_precip_hist_annual_mdb, 
                                           ff5_precip_hist_annual_mdb, ff6_precip_hist_annual_mdb, 
                                           ff7_precip_hist_annual_mdb, ff8_precip_hist_annual_mdb, 
                                           ff9_precip_hist_annual_mdb, ff10_precip_hist_annual_mdb,
                                           ff11_precip_hist_annual_mdb, ff12_precip_hist_annual_mdb,
                                           ff13_precip_hist_annual_mdb], dim='en')
ff_all_precip_hist_annual_mdb_mean = ff_all_precip_hist_annual_mdb.mean(dim='en')
ff_all_precip_hist_annual_mdb_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_hist_annual_mdb_mean, 1900,2000)

giss_all_precip_hist_annual_mdb = xr.concat([giss_21_precip_hist_annual_mdb, giss_22_precip_hist_annual_mdb, giss_23_precip_hist_annual_mdb,
                                             giss_24_precip_hist_annual_mdb, giss_25_precip_hist_annual_mdb, giss_26_precip_hist_annual_mdb,
                                             giss_27_precip_hist_annual_mdb, giss_28_precip_hist_annual_mdb], dim='en')
giss_all_precip_hist_annual_mdb_mean = giss_all_precip_hist_annual_mdb.mean(dim='en')
giss_all_precip_hist_annual_mdb_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_hist_annual_mdb_mean, 1900, 2000)





In [51]:
hist_mdb_dir = '../files/historical_1900/mdb'
if not os.path.exists(hist_mdb_dir):
    print("... Creating %s now "  % hist_mdb_dir)
    os.makedirs(hist_mdb_dir)

... Creating ../files/historical_1900/mdb now 


In [52]:
save_netcdf_compression(ff1_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff1_precip_hist_annual_mdb')
save_netcdf_compression(ff2_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff2_precip_hist_annual_mdb')
save_netcdf_compression(ff3_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff3_precip_hist_annual_mdb')
save_netcdf_compression(ff4_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff4_precip_hist_annual_mdb')
save_netcdf_compression(ff5_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff5_precip_hist_annual_mdb')
save_netcdf_compression(ff6_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff6_precip_hist_annual_mdb')
save_netcdf_compression(ff7_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff7_precip_hist_annual_mdb')
save_netcdf_compression(ff8_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff8_precip_hist_annual_mdb')
save_netcdf_compression(ff9_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff9_precip_hist_annual_mdb')
save_netcdf_compression(ff10_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff10_precip_hist_annual_mdb')
save_netcdf_compression(ff11_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff11_precip_hist_annual_mdb')
save_netcdf_compression(ff12_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff12_precip_hist_annual_mdb')
save_netcdf_compression(ff13_precip_hist_annual_mdb_rc, hist_mdb_dir, 'cesmlme-ff13_precip_hist_annual_mdb')

save_netcdf_compression(bcc_precip_hist_annual_mdb_rc, hist_mdb_dir, 'bcc_precip_hist_annual_mdb')
save_netcdf_compression(ccsm4_precip_hist_annual_mdb_rc, hist_mdb_dir, 'ccsm4_precip_hist_annual_mdb')
save_netcdf_compression(csiro_mk3l_precip_hist_annual_mdb_rc, hist_mdb_dir, 'csiro_mk3l_precip_hist_annual_mdb')
save_netcdf_compression(fgoals_gl_precip_hist_annual_mdb_rc, hist_mdb_dir, 'fgoals_gl_precip_hist_annual_mdb')
save_netcdf_compression(fgoals_s2_precip_hist_annual_mdb_rc, hist_mdb_dir, 'fgoals_s2_precip_hist_annual_mdb')
save_netcdf_compression(giss_21_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_21_precip_hist_annual_mdb')
save_netcdf_compression(giss_22_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_22_precip_hist_annual_mdb')
save_netcdf_compression(giss_23_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_23_precip_hist_annual_mdb')
save_netcdf_compression(giss_24_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_24_precip_hist_annual_mdb')
save_netcdf_compression(giss_25_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_25_precip_hist_annual_mdb')
save_netcdf_compression(giss_26_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_26_precip_hist_annual_mdb')
save_netcdf_compression(giss_27_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_27_precip_hist_annual_mdb')
save_netcdf_compression(giss_28_precip_hist_annual_mdb_rc, hist_mdb_dir, 'giss_28_precip_hist_annual_mdb')
save_netcdf_compression(hadcm3_precip_hist_annual_mdb_rc, hist_mdb_dir, 'hadcm3_precip_hist_annual_mdb')
save_netcdf_compression(ipsl_precip_hist_annual_mdb_rc, hist_mdb_dir, 'ipsl_precip_hist_annual_mdb')
save_netcdf_compression(miroc_precip_hist_annual_mdb_rc, hist_mdb_dir, 'miroc_precip_hist_annual_mdb')
save_netcdf_compression(mpi_precip_hist_annual_mdb_rc, hist_mdb_dir, 'mpi_precip_hist_annual_mdb')
save_netcdf_compression(mri_precip_hist_annual_mdb_rc, hist_mdb_dir, 'mri_precip_hist_annual_mdb')
save_netcdf_compression(ff_all_precip_hist_annual_mdb_mean_rc, hist_mdb_dir, 'ff_all_precip_hist_annual_mdb')
save_netcdf_compression(giss_all_precip_hist_annual_mdb_mean_rc, hist_mdb_dir, 'giss_all_precip_hist_annual_mdb')


save_netcdf_compression(awap_masked_annual_mdb_rc, hist_mdb_dir, 'awap_masked_annual_mdb')


[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

### last millennium

In [21]:
ff1_precip_lm_annual_mdb        = get_mdb_mean(ff1_precip_lm_annual_aus)
ff2_precip_lm_annual_mdb        = get_mdb_mean(ff2_precip_lm_annual_aus)
ff3_precip_lm_annual_mdb        = get_mdb_mean(ff3_precip_lm_annual_aus)
ff4_precip_lm_annual_mdb        = get_mdb_mean(ff4_precip_lm_annual_aus)
ff5_precip_lm_annual_mdb        = get_mdb_mean(ff5_precip_lm_annual_aus)
ff6_precip_lm_annual_mdb        = get_mdb_mean(ff6_precip_lm_annual_aus)
ff7_precip_lm_annual_mdb        = get_mdb_mean(ff7_precip_lm_annual_aus)
ff8_precip_lm_annual_mdb        = get_mdb_mean(ff8_precip_lm_annual_aus)
ff9_precip_lm_annual_mdb        = get_mdb_mean(ff9_precip_lm_annual_aus)
ff10_precip_lm_annual_mdb       = get_mdb_mean(ff10_precip_lm_annual_aus)
ff11_precip_lm_annual_mdb       = get_mdb_mean(ff11_precip_lm_annual_aus)
ff12_precip_lm_annual_mdb       = get_mdb_mean(ff12_precip_lm_annual_aus)
ff13_precip_lm_annual_mdb       = get_mdb_mean(ff13_precip_lm_annual_aus)
bcc_precip_lm_annual_mdb        = get_mdb_mean(bcc_precip_lm_annual_aus)
ccsm4_precip_lm_annual_mdb      = get_mdb_mean(ccsm4_precip_lm_annual_aus)
csiro_mk3l_precip_lm_annual_mdb = get_mdb_mean(csiro_mk3l_precip_lm_annual_aus)
fgoals_gl_precip_lm_annual_mdb  = get_mdb_mean(fgoals_gl_precip_lm_annual_aus)
fgoals_s2_precip_lm_annual_mdb  = get_mdb_mean(fgoals_s2_precip_lm_annual_aus)
giss_21_precip_lm_annual_mdb    = get_mdb_mean(giss_21_precip_lm_annual_aus)
giss_22_precip_lm_annual_mdb    = get_mdb_mean(giss_22_precip_lm_annual_aus)
giss_23_precip_lm_annual_mdb    = get_mdb_mean(giss_23_precip_lm_annual_aus)
giss_24_precip_lm_annual_mdb    = get_mdb_mean(giss_24_precip_lm_annual_aus)
giss_25_precip_lm_annual_mdb    = get_mdb_mean(giss_25_precip_lm_annual_aus)
giss_26_precip_lm_annual_mdb    = get_mdb_mean(giss_26_precip_lm_annual_aus)
giss_27_precip_lm_annual_mdb    = get_mdb_mean(giss_27_precip_lm_annual_aus)
giss_28_precip_lm_annual_mdb    = get_mdb_mean(giss_28_precip_lm_annual_aus)
hadcm3_precip_lm_annual_mdb     = get_mdb_mean(hadcm3_precip_lm_annual_aus)
ipsl_precip_lm_annual_mdb       = get_mdb_mean(ipsl_precip_lm_annual_aus)
miroc_precip_lm_annual_mdb      = get_mdb_mean(miroc_precip_lm_annual_aus)
mpi_precip_lm_annual_mdb        = get_mdb_mean(mpi_precip_lm_annual_aus)
mri_precip_lm_annual_mdb        = get_mdb_mean(mri_precip_lm_annual_aus)

In [53]:
ff1_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff1_precip_lm_annual_mdb       , 1900, 2000)
ff2_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff2_precip_lm_annual_mdb       , 1900, 2000)
ff3_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff3_precip_lm_annual_mdb       , 1900, 2000)
ff4_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff4_precip_lm_annual_mdb       , 1900, 2000)
ff5_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff5_precip_lm_annual_mdb       , 1900, 2000)
ff6_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff6_precip_lm_annual_mdb       , 1900, 2000)
ff7_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff7_precip_lm_annual_mdb       , 1900, 2000)
ff8_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff8_precip_lm_annual_mdb       , 1900, 2000)
ff9_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(ff9_precip_lm_annual_mdb       , 1900, 2000)
ff10_precip_lm_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff10_precip_lm_annual_mdb      , 1900, 2000)
ff11_precip_lm_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff11_precip_lm_annual_mdb      , 1900, 2000)
ff12_precip_lm_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff12_precip_lm_annual_mdb      , 1900, 2000)
ff13_precip_lm_annual_mdb_rc       = droughts_lastmillennium_fromyear(ff13_precip_lm_annual_mdb      , 1900, 2000)
bcc_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(bcc_precip_lm_annual_mdb       , 1900, 2000)
ccsm4_precip_lm_annual_mdb_rc      = droughts_lastmillennium_fromyear(ccsm4_precip_lm_annual_mdb     , 1900, 2000)
csiro_mk3l_precip_lm_annual_mdb_rc = droughts_lastmillennium_fromyear(csiro_mk3l_precip_lm_annual_mdb, 1900, 2000)
fgoals_gl_precip_lm_annual_mdb_rc  = droughts_lastmillennium_fromyear(fgoals_gl_precip_lm_annual_mdb , 1900, 2000)
fgoals_s2_precip_lm_annual_mdb_rc  = droughts_lastmillennium_fromyear(fgoals_s2_precip_lm_annual_mdb , 1900, 2000)
giss_21_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_21_precip_lm_annual_mdb   , 1900, 2000)
giss_22_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_22_precip_lm_annual_mdb   , 1900, 2000)
giss_23_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_23_precip_lm_annual_mdb   , 1900, 2000)
giss_24_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_24_precip_lm_annual_mdb   , 1900, 2000)
giss_25_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_25_precip_lm_annual_mdb   , 1900, 2000)
giss_26_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_26_precip_lm_annual_mdb   , 1900, 2000)
giss_27_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_27_precip_lm_annual_mdb   , 1900, 2000)
giss_28_precip_lm_annual_mdb_rc    = droughts_lastmillennium_fromyear(giss_28_precip_lm_annual_mdb   , 1900, 2000)
hadcm3_precip_lm_annual_mdb_rc     = droughts_lastmillennium_fromyear(hadcm3_precip_lm_annual_mdb    , 1900, 2000)
ipsl_precip_lm_annual_mdb_rc       = droughts_lastmillennium_fromyear(ipsl_precip_lm_annual_mdb      , 1900, 2000)
miroc_precip_lm_annual_mdb_rc      = droughts_lastmillennium_fromyear(miroc_precip_lm_annual_mdb     , 1900, 2000)
mpi_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(mpi_precip_lm_annual_mdb       , 1900, 2000)
mri_precip_lm_annual_mdb_rc        = droughts_lastmillennium_fromyear(mri_precip_lm_annual_mdb       , 1900, 2000)

In [54]:
ff_all_precip_lm_annual_mdb = xr.concat([ff1_precip_lm_annual_mdb, ff2_precip_lm_annual_mdb,
                                                    ff3_precip_lm_annual_mdb, ff4_precip_lm_annual_mdb, 
                                                    ff5_precip_lm_annual_mdb, ff6_precip_lm_annual_mdb, 
                                                    ff7_precip_lm_annual_mdb, ff8_precip_lm_annual_mdb, 
                                                    ff9_precip_lm_annual_mdb, ff10_precip_lm_annual_mdb,
                                                    ff11_precip_lm_annual_mdb, ff12_precip_lm_annual_mdb,
                                                    ff13_precip_lm_annual_mdb], dim='en')
ff_all_precip_lm_annual_mdb_mean = ff_all_precip_lm_annual_mdb.mean(dim='en')
ff_all_precip_lm_annual_mdb_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_lm_annual_mdb_mean, 1900,2000)

giss_all_precip_lm_annual_mdb = xr.concat([giss_21_precip_lm_annual_mdb,
                                                      giss_22_precip_lm_annual_mdb,
                                                      giss_23_precip_lm_annual_mdb,
                                                      giss_24_precip_lm_annual_mdb,
                                                      giss_25_precip_lm_annual_mdb,
                                                      giss_26_precip_lm_annual_mdb,
                                                      giss_27_precip_lm_annual_mdb,
                                                      giss_28_precip_lm_annual_mdb], dim='en')
giss_all_precip_lm_annual_mdb_mean = giss_all_precip_lm_annual_mdb.mean(dim='en')
giss_all_precip_lm_annual_mdb_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_lm_annual_mdb_mean, 1900, 2000)

In [56]:
lm_mdb_dir = '../files/lastmillennium_threshold_1900-2000/mdb'
if not os.path.exists(lm_mdb_dir):
    print("... Creating %s now "  % lm_mdb_dir)
    os.makedirs(lm_mdb_dir)

... Creating ../files/lastmillennium_threshold_1900-2000/mdb now 


In [57]:
save_netcdf_compression(ff1_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff1_precip_lm_annual_mdb')
save_netcdf_compression(ff2_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff2_precip_lm_annual_mdb')
save_netcdf_compression(ff3_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff3_precip_lm_annual_mdb')
save_netcdf_compression(ff4_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff4_precip_lm_annual_mdb')
save_netcdf_compression(ff5_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff5_precip_lm_annual_mdb')
save_netcdf_compression(ff6_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff6_precip_lm_annual_mdb')
save_netcdf_compression(ff7_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff7_precip_lm_annual_mdb')
save_netcdf_compression(ff8_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff8_precip_lm_annual_mdb')
save_netcdf_compression(ff9_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff9_precip_lm_annual_mdb')
save_netcdf_compression(ff10_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff10_precip_lm_annual_mdb')
save_netcdf_compression(ff11_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff11_precip_lm_annual_mdb')
save_netcdf_compression(ff12_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff12_precip_lm_annual_mdb')
save_netcdf_compression(ff13_precip_lm_annual_mdb_rc, lm_mdb_dir, 'cesmlme-ff13_precip_lm_annual_mdb')

save_netcdf_compression(bcc_precip_lm_annual_mdb_rc, lm_mdb_dir, 'bcc_precip_lm_annual_mdb')
save_netcdf_compression(ccsm4_precip_lm_annual_mdb_rc, lm_mdb_dir, 'ccsm4_precip_lm_annual_mdb')
save_netcdf_compression(csiro_mk3l_precip_lm_annual_mdb_rc, lm_mdb_dir, 'csiro_mk3l_precip_lm_annual_mdb')
save_netcdf_compression(fgoals_gl_precip_lm_annual_mdb_rc,  lm_mdb_dir, 'fgoals_gl_precip_lm_annual_mdb')
save_netcdf_compression(fgoals_s2_precip_lm_annual_mdb_rc,  lm_mdb_dir, 'fgoals_s2_precip_lm_annual_mdb')
save_netcdf_compression(giss_21_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_21_precip_lm_annual_mdb')
save_netcdf_compression(giss_22_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_22_precip_lm_annual_mdb')
save_netcdf_compression(giss_23_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_23_precip_lm_annual_mdb')
save_netcdf_compression(giss_24_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_24_precip_lm_annual_mdb')
save_netcdf_compression(giss_25_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_25_precip_lm_annual_mdb')
save_netcdf_compression(giss_26_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_26_precip_lm_annual_mdb')
save_netcdf_compression(giss_27_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_27_precip_lm_annual_mdb')
save_netcdf_compression(giss_28_precip_lm_annual_mdb_rc, lm_mdb_dir, 'giss_28_precip_lm_annual_mdb')
save_netcdf_compression(hadcm3_precip_lm_annual_mdb_rc, lm_mdb_dir, 'hadcm3_precip_lm_annual_mdb')
save_netcdf_compression(ipsl_precip_lm_annual_mdb_rc,   lm_mdb_dir, 'ipsl_precip_lm_annual_mdb')
save_netcdf_compression(miroc_precip_lm_annual_mdb_rc,  lm_mdb_dir, 'miroc_precip_lm_annual_mdb')
save_netcdf_compression(mpi_precip_lm_annual_mdb_rc, lm_mdb_dir, 'mpi_precip_lm_annual_mdb')
save_netcdf_compression(mri_precip_lm_annual_mdb_rc, lm_mdb_dir, 'mri_precip_lm_annual_mdb')

save_netcdf_compression(giss_all_precip_lm_annual_mdb_mean_rc, lm_mdb_dir, 'giss_all_precip_lm_annual_mdb_rc')
save_netcdf_compression(ff_all_precip_lm_annual_mdb_mean_rc,   lm_mdb_dir, 'cesmlme-ff_all_precip_lm_annual_mdb_rc')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

### control

In [11]:
bcc_annual_cntl_mdb         = get_mdb_mean(bcc_annual_cntl_aus)
ccsm4_annual_cntl_mdb       = get_mdb_mean(ccsm4_annual_cntl_aus)
csiro_mk3l_annual_cntl_mdb  = get_mdb_mean(csiro_mk3l_annual_cntl_aus)
fgoals_s2_annual_cntl_mdb   = get_mdb_mean(fgoals_s2_annual_cntl_aus)
giss_2_annual_cntl_mdb      = get_mdb_mean(giss_2_annual_cntl_aus)
hadcm3_annual_cntl_mdb      = get_mdb_mean(hadcm3_annual_cntl_aus)
ipsl_annual_cntl_mdb        = get_mdb_mean(ipsl_annual_cntl_aus)
miroc_annual_cntl_mdb       = get_mdb_mean(miroc_annual_cntl_aus)
mpi_annual_cntl_mdb         = get_mdb_mean(mpi_annual_cntl_aus)
mri_annual_cntl_mdb         = get_mdb_mean(mri_annual_cntl_aus)
giss_3_annual_cntl_mdb      = get_mdb_mean(giss_3_annual_cntl_aus)
giss_1_annual_cntl_mdb      = get_mdb_mean(giss_1_annual_cntl_aus)
giss_41_annual_cntl_mdb     = get_mdb_mean(giss_41_annual_cntl_aus)


In [20]:
bcc_annual_cntl_mdb_rc        = droughts_lastmillennium_fromyear(bcc_annual_cntl_mdb, bcc_annual_cntl_mdb.year[0].values, bcc_annual_cntl_mdb.year[-1].values)
ccsm4_annual_cntl_mdb_rc      = droughts_lastmillennium_fromyear(ccsm4_annual_cntl_mdb, ccsm4_annual_cntl_mdb.year[0].values, ccsm4_annual_cntl_mdb.year[-1].values)
csiro_mk3l_annual_cntl_mdb_rc = droughts_lastmillennium_fromyear(csiro_mk3l_annual_cntl_mdb, csiro_mk3l_annual_cntl_mdb.year[0].values, csiro_mk3l_annual_cntl_mdb.year[-1].values)
fgoals_s2_annual_cntl_mdb_rc  = droughts_lastmillennium_fromyear(fgoals_s2_annual_cntl_mdb, fgoals_s2_annual_cntl_mdb.year[0].values, fgoals_s2_annual_cntl_mdb.year[-1].values)
giss_2_annual_cntl_mdb_rc     = droughts_lastmillennium_fromyear(giss_2_annual_cntl_mdb, giss_2_annual_cntl_mdb.year[0].values, giss_2_annual_cntl_mdb.year[-1].values)
hadcm3_annual_cntl_mdb_rc     = droughts_lastmillennium_fromyear(hadcm3_annual_cntl_mdb, hadcm3_annual_cntl_mdb.year[0].values, hadcm3_annual_cntl_mdb.year[-1].values)
ipsl_annual_cntl_mdb_rc       = droughts_lastmillennium_fromyear(ipsl_annual_cntl_mdb, ipsl_annual_cntl_mdb.year[0].values, ipsl_annual_cntl_mdb.year[-1].values)
miroc_annual_cntl_mdb_rc      = droughts_lastmillennium_fromyear(miroc_annual_cntl_mdb, miroc_annual_cntl_mdb.year[0].values, miroc_annual_cntl_mdb.year[-1].values)
mpi_annual_cntl_mdb_rc        = droughts_lastmillennium_fromyear(mpi_annual_cntl_mdb, mpi_annual_cntl_mdb.year[0].values, mpi_annual_cntl_mdb.year[-1].values)
mri_annual_cntl_mdb_rc        = droughts_lastmillennium_fromyear(mri_annual_cntl_mdb, mri_annual_cntl_mdb.year[0].values, mri_annual_cntl_mdb.year[-1].values)
giss_3_annual_cntl_mdb_rc     = droughts_lastmillennium_fromyear(giss_3_annual_cntl_mdb, giss_3_annual_cntl_mdb.year[0].values, giss_3_annual_cntl_mdb.year[-1].values)
giss_1_annual_cntl_mdb_rc     = droughts_lastmillennium_fromyear(giss_1_annual_cntl_mdb, giss_1_annual_cntl_mdb.year[0].values, giss_1_annual_cntl_mdb.year[-1].values)
giss_41_annual_cntl_mdb_rc    = droughts_lastmillennium_fromyear(giss_41_annual_cntl_mdb, giss_41_annual_cntl_mdb.year[0].values, giss_41_annual_cntl_mdb.year[-1].values)


In [21]:
ctrl_mdb_dir = '../files/control/mdb'
if not os.path.exists(ctrl_mdb_dir):
    print("... Creating %s now "  % ctrl_mdb_dir)
    os.makedirs(ctrl_mdb_dir)

... Creating ../files/control/mdb now 


In [22]:
save_netcdf_compression(bcc_annual_cntl_mdb_rc, ctrl_mdb_dir, 'bcc_annual_cntl_mdb')
save_netcdf_compression(ccsm4_annual_cntl_mdb_rc, ctrl_mdb_dir, 'ccsm4_annual_cntl_mdb')
save_netcdf_compression(csiro_mk3l_annual_cntl_mdb_rc, ctrl_mdb_dir, 'csiro_mk3l_annual_cntl_mdb')
save_netcdf_compression(fgoals_s2_annual_cntl_mdb_rc, ctrl_mdb_dir, 'fgoals_s2_annual_cntl_mdb')
save_netcdf_compression(giss_2_annual_cntl_mdb_rc, ctrl_mdb_dir, 'giss_2_annual_cntl_mdb')
save_netcdf_compression(hadcm3_annual_cntl_mdb_rc, ctrl_mdb_dir, 'hadcm3_annual_cntl_mdb')
save_netcdf_compression(ipsl_annual_cntl_mdb_rc, ctrl_mdb_dir, 'ipsl_annual_cntl_mdb')
save_netcdf_compression(miroc_annual_cntl_mdb_rc, ctrl_mdb_dir, 'miroc_annual_cntl_mdb')
save_netcdf_compression(mpi_annual_cntl_mdb_rc, ctrl_mdb_dir, 'mpi_annual_cntl_mdb')
save_netcdf_compression(mri_annual_cntl_mdb_rc, ctrl_mdb_dir, 'mri_annual_cntl_mdb')
save_netcdf_compression(giss_3_annual_cntl_mdb_rc, ctrl_mdb_dir, 'giss_3_annual_cntl_mdb')
save_netcdf_compression(giss_1_annual_cntl_mdb_rc, ctrl_mdb_dir, 'giss_1_annual_cntl_mdb')
save_netcdf_compression(giss_41_annual_cntl_mdb_rc, ctrl_mdb_dir, 'giss_41_annual_cntl_mdb')


[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


---

## HAVE NOT MODIFIED BELOW HERE

In [ ]:
input_dir_lm = 'proc_lastmillennium_threshold_1900-2000/'

In [ ]:
ff1_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff1_precip_lm_annual_mdb.nc' % input_dir_lm)
ff2_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff2_precip_lm_annual_mdb.nc' % input_dir_lm)
ff3_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff3_precip_lm_annual_mdb.nc' % input_dir_lm)
ff4_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff4_precip_lm_annual_mdb.nc' % input_dir_lm)
ff5_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff5_precip_lm_annual_mdb.nc' % input_dir_lm)
ff6_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff6_precip_lm_annual_mdb.nc' % input_dir_lm)
ff7_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff7_precip_lm_annual_mdb.nc' % input_dir_lm)
ff8_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff8_precip_lm_annual_mdb.nc' % input_dir_lm)
ff9_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff9_precip_lm_annual_mdb.nc' % input_dir_lm)
ff10_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff10_precip_lm_annual_mdb.nc' % input_dir_lm)
ff11_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff11_precip_lm_annual_mdb.nc' % input_dir_lm)
ff12_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff12_precip_lm_annual_mdb.nc' % input_dir_lm)
ff13_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ff13_precip_lm_annual_mdb.nc' % input_dir_lm)

bcc_precip_lm_annual_mdb_rc = xr.open_dataset('%s/bcc_precip_lm_annual_mdb.nc' % input_dir_lm)
ccsm4_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ccsm4_precip_lm_annual_mdb.nc' % input_dir_lm)
csiro_mk3l_precip_lm_annual_mdb_rc= xr.open_dataset('%s/csiro_mk3l_precip_lm_annual_mdb.nc' % input_dir_lm)
fgoals_gl_precip_lm_annual_mdb_rc = xr.open_dataset('%s/fgoals_gl_precip_lm_annual_mdb.nc' % input_dir_lm)
fgoals_s2_precip_lm_annual_mdb_rc = xr.open_dataset('%s/fgoals_s2_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_21_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_21_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_22_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_22_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_23_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_23_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_24_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_24_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_25_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_25_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_26_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_26_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_27_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_27_precip_lm_annual_mdb.nc' % input_dir_lm)
giss_28_precip_lm_annual_mdb_rc= xr.open_dataset('%s/giss_28_precip_lm_annual_mdb.nc' % input_dir_lm)
hadcm3_precip_lm_annual_mdb_rc = xr.open_dataset('%s/hadcm3_precip_lm_annual_mdb.nc' % input_dir_lm)
ipsl_precip_lm_annual_mdb_rc = xr.open_dataset('%s/ipsl_precip_lm_annual_mdb.nc' % input_dir_lm)
miroc_precip_lm_annual_mdb_rc= xr.open_dataset('%s/miroc_precip_lm_annual_mdb.nc' % input_dir_lm)
mpi_precip_lm_annual_mdb_rc = xr.open_dataset('%s/mpi_precip_lm_annual_mdb.nc' % input_dir_lm)
mri_precip_lm_annual_mdb_rc = xr.open_dataset('%s/mri_precip_lm_annual_mdb.nc' % input_dir_lm)

giss_all_precip_lm_annual_mdb_mean_rc = xr.open_dataset('%s/giss_all_precip_lm_annual_mdb_rc.nc' % input_dir_lm)
ff_all_precip_lm_annual_mdb_mean_rc = xr.open_dataset('%s/ff_all_precip_lm_annual_mdb_rc.nc' % input_dir_lm)

In [15]:
ds_mdb_prect_lm = bcc_precip_lm_annual_mdb_rc.PRECT_mm
ds_mdb_prect_lm = ds_mdb_prect_lm.to_dataset()
ds_mdb_prect_lm['bcc'] = ds_mdb_prect_lm.PRECT_mm

ds_mdb_prect_lm['ccsm4'] = ccsm4_precip_lm_annual_mdb_rc.PRECT_mm
ds_mdb_prect_lm['csiro_mk3l'] = csiro_mk3l_precip_lm_annual_mdb_rc.PRECT_mm
ds_mdb_prect_lm['fgoals_gl'] = fgoals_gl_precip_lm_annual_mdb_rc.PRECT_mm 
ds_mdb_prect_lm['fgoals_s2'] = fgoals_s2_precip_lm_annual_mdb_rc.PRECT_mm 
ds_mdb_prect_lm['giss_21'] = giss_21_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['giss_22'] = giss_22_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['giss_23'] = giss_23_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['giss_24'] = giss_24_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['giss_25'] = giss_25_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['giss_26'] = giss_26_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['giss_27'] = giss_27_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['giss_28'] = giss_28_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['hadcm3'] = hadcm3_precip_lm_annual_mdb_rc.PRECT_mm    
ds_mdb_prect_lm['ipsl'] = ipsl_precip_lm_annual_mdb_rc.PRECT_mm      
ds_mdb_prect_lm['miroc'] = miroc_precip_lm_annual_mdb_rc.PRECT_mm     
ds_mdb_prect_lm['mpi'] = mpi_precip_lm_annual_mdb_rc.PRECT_mm       
ds_mdb_prect_lm['mri'] = mri_precip_lm_annual_mdb_rc.PRECT_mm
ds_mdb_prect_lm['ff1'] = ff1_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff2'] = ff2_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff3'] = ff3_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff4'] = ff4_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff5'] = ff5_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff6'] = ff6_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff7'] = ff7_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff8'] = ff8_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff9'] = ff9_precip_lm_annual_mdb_rc.PRECT_mm   
ds_mdb_prect_lm['ff10'] = ff10_precip_lm_annual_mdb_rc.PRECT_mm  
ds_mdb_prect_lm['ff11'] = ff11_precip_lm_annual_mdb_rc.PRECT_mm  
ds_mdb_prect_lm['ff12'] = ff12_precip_lm_annual_mdb_rc.PRECT_mm  
ds_mdb_prect_lm['ff13'] = ff13_precip_lm_annual_mdb_rc.PRECT_mm

ds_mdb_prect_lm = ds_mdb_prect_lm.drop(('PRECT_mm'))

save_netcdf_compression(ds_mdb_prect_lm, input_dir_lm, 'pmip3_models_precip_lm_mdb')

[########################################] | 100% Completed |  0.1s


In [7]:
# bcc_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.bcc, 'bcc')
# save_netcdf_compression(bcc_boot_mdb, 'bootstrap_everyyear/mdb', 'bcc_boot_mdb')

# ccsm4_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ccsm4, 'ccsm4')
# save_netcdf_compression(ccsm4_boot_mdb, 'bootstrap_everyyear/mdb', 'ccsm4_boot_mdb')

# csiro_mk3l_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.csiro_mk3l, 'csiro_mk3l')
# save_netcdf_compression(csiro_mk3l_boot_mdb, 'bootstrap_everyyear/mdb', 'csiro_mk3l_boot_mdb')

# fgoals_gl_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.fgoals_gl, 'fgoals_gl')
# save_netcdf_compression(fgoals_gl_boot_mdb, 'bootstrap_everyyear/mdb', 'fgoals_gl_boot_mdb')

# fgoals_s2_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.fgoals_s2, 'fgoals_s2')
# save_netcdf_compression(fgoals_s2_boot_mdb, 'bootstrap_everyyear/mdb', 'fgoals_s2_boot_mdb')

# giss_21_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_21, 'giss_21')
# save_netcdf_compression(giss_21_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_21_boot_mdb')

# giss_22_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_22, 'giss_22')
# save_netcdf_compression(giss_22_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_22_boot_mdb')

# giss_23_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_23, 'giss_23')
# save_netcdf_compression(giss_23_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_23_boot_mdb')

# giss_24_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_24, 'giss_24')
# save_netcdf_compression(giss_24_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_24_boot_mdb')

# giss_25_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_25, 'giss_25')
# save_netcdf_compression(giss_25_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_25_boot_mdb')

# giss_26_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_26, 'giss_26')
# save_netcdf_compression(giss_26_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_26_boot_mdb')

# giss_27_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_27, 'giss_27')
# save_netcdf_compression(giss_27_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_27_boot_mdb')

# giss_28_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.giss_28, 'giss_28')
# save_netcdf_compression(giss_28_boot_mdb, 'bootstrap_everyyear/mdb', 'giss_28_boot_mdb')

# hadcm3_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.hadcm3, 'hadcm3')
# save_netcdf_compression(hadcm3_boot_mdb, 'bootstrap_everyyear/mdb', 'hadcm3_boot_mdb')

# ipsl_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ipsl, 'ipsl')
# save_netcdf_compression(ipsl_boot_mdb, 'bootstrap_everyyear/mdb', 'ipsl_boot_mdb')

# miroc_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.miroc, 'miroc')
# save_netcdf_compression(miroc_boot_mdb, 'bootstrap_everyyear/mdb', 'miroc_boot_mdb')

# mpi_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.mpi, 'mpi')
# save_netcdf_compression(mpi_boot_mdb, 'bootstrap_everyyear/mdb', 'mpi_boot_mdb')

# mri_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.mri, 'mri')
# save_netcdf_compression(mri_boot_mdb, 'bootstrap_everyyear/mdb', 'mri_boot_mdb')

# ff1_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff1, 'ff1')
# save_netcdf_compression(ff1_boot_mdb, 'bootstrap_everyyear/mdb', 'ff1_boot_mdb')

# ff2_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff2, 'ff2')
# save_netcdf_compression(ff2_boot_mdb, 'bootstrap_everyyear/mdb', 'ff2_boot_mdb')

# ff3_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff3, 'ff3')
# save_netcdf_compression(ff3_boot_mdb, 'bootstrap_everyyear/mdb', 'ff3_boot_mdb')

# ff4_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff4, 'ff4')
# save_netcdf_compression(ff4_boot_mdb, 'bootstrap_everyyear/mdb', 'ff4_boot_mdb')

# ff5_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff5, 'ff5')
# save_netcdf_compression(ff5_boot_mdb, 'bootstrap_everyyear/mdb', 'ff5_boot_mdb')

# ff6_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff6, 'ff6')
# save_netcdf_compression(ff6_boot_mdb, 'bootstrap_everyyear/mdb', 'ff6_boot_mdb')

# ff7_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff7, 'ff7')
# save_netcdf_compression(ff7_boot_mdb, 'bootstrap_everyyear/mdb', 'ff7_boot_mdb')

# ff8_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff8, 'ff8')
# save_netcdf_compression(ff8_boot_mdb, 'bootstrap_everyyear/mdb', 'ff8_boot_mdb')

# ff9_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff9, 'ff9')
# save_netcdf_compression(ff9_boot_mdb, 'bootstrap_everyyear/mdb', 'ff9_boot_mdb')

# ff10_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff10, 'ff10')
# save_netcdf_compression(ff10_boot_mdb, 'bootstrap_everyyear/mdb', 'ff10_boot_mdb')

# ff11_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff11, 'ff11')
# save_netcdf_compression(ff11_boot_mdb, 'bootstrap_everyyear/mdb', 'ff11_boot_mdb')

# ff12_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff12, 'ff12')
# save_netcdf_compression(ff12_boot_mdb, 'bootstrap_everyyear/mdb', 'ff12_boot_mdb')

# ff13_boot_mdb = bootstrap_years_101(ds_mdb_prect_lm.ff13, 'ff13')
# save_netcdf_compression(ff13_boot_mdb, 'bootstrap_everyyear/mdb', 'ff13_boot_mdb')

# ff_all_precip_lm_annual_mdb_mean_rc_boots = bootstrap_years_101(ff_all_precip_lm_annual_mdb_mean_rc.PRECT_mm, 'PRECT_mm')
# save_netcdf_compression(ff_all_precip_lm_annual_mdb_mean_rc_boots, 'bootstrap_everyyear/mdb', 'ff_all_boot_mdb')

# giss_all_precip_lm_annual_mdb_mean_rc_boots = bootstrap_years_101(giss_all_precip_lm_annual_mdb_mean_rc.PRECT_mm, 'PRECT_mm')
# save_netcdf_compression(giss_all_precip_lm_annual_mdb_mean_rc_boots, 'bootstrap_everyyear/mdb', 'giss_all_boot_mdb')

## southern Aus

In [10]:
def get_southern_aus_mean(ds):
    ds_southern = ds.where(ds.lat <= -26, drop=True)
    ds_southern_mean = climate_xr_funcs.weighted_mean_CAM(ds_southern)    
    return ds_southern_mean

In [20]:
ff1_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff1_precip_hist_annual_aus)
ff2_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff2_precip_hist_annual_aus)
ff3_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff3_precip_hist_annual_aus)
ff4_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff4_precip_hist_annual_aus)
ff5_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff5_precip_hist_annual_aus)
ff6_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff6_precip_hist_annual_aus)
ff7_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff7_precip_hist_annual_aus)
ff8_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff8_precip_hist_annual_aus)
ff9_precip_hist_annual_southern_aus       = get_southern_aus_mean(ff9_precip_hist_annual_aus)
ff10_precip_hist_annual_southern_aus      = get_southern_aus_mean(ff10_precip_hist_annual_aus)
ff11_precip_hist_annual_southern_aus      = get_southern_aus_mean(ff11_precip_hist_annual_aus)
ff12_precip_hist_annual_southern_aus      = get_southern_aus_mean(ff12_precip_hist_annual_aus)
ff13_precip_hist_annual_southern_aus      = get_southern_aus_mean(ff13_precip_hist_annual_aus)

bcc_precip_hist_annual_southern_aus        = get_southern_aus_mean(bcc_precip_hist_annual_aus)
ccsm4_precip_hist_annual_southern_aus      = get_southern_aus_mean(ccsm4_precip_hist_annual_aus)
csiro_mk3l_precip_hist_annual_southern_aus = get_southern_aus_mean(csiro_mk3l_precip_hist_annual_aus)
fgoals_gl_precip_hist_annual_southern_aus  = get_southern_aus_mean(fgoals_gl_precip_hist_annual_aus)
fgoals_s2_precip_hist_annual_southern_aus  = get_southern_aus_mean(fgoals_s2_precip_hist_annual_aus)
giss_21_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_21_precip_hist_annual_aus)
giss_22_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_22_precip_hist_annual_aus)
giss_23_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_23_precip_hist_annual_aus)
giss_24_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_24_precip_hist_annual_aus)
giss_25_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_25_precip_hist_annual_aus)
giss_26_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_26_precip_hist_annual_aus)
giss_27_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_27_precip_hist_annual_aus)
giss_28_precip_hist_annual_southern_aus    = get_southern_aus_mean(giss_28_precip_hist_annual_aus)
hadcm3_precip_hist_annual_southern_aus     = get_southern_aus_mean(hadcm3_precip_hist_annual_aus)
ipsl_precip_hist_annual_southern_aus       = get_southern_aus_mean(ipsl_precip_hist_annual_aus)
miroc_precip_hist_annual_southern_aus      = get_southern_aus_mean(miroc_precip_hist_annual_aus)
mpi_precip_hist_annual_southern_aus        = get_southern_aus_mean(mpi_precip_hist_annual_aus)
mri_precip_hist_annual_southern_aus        = get_southern_aus_mean(mri_precip_hist_annual_aus)

In [11]:
awap_masked_annual_southern_aus = get_southern_aus_mean(awap_gf_annual)

In [13]:
awap_masked_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(awap_masked_annual_southern_aus, 1900, 2000)    


save_netcdf_compression(awap_masked_annual_southern_aus_rc, input_dir_hist, 'awap_masked_annual_southern_aus')



[########################################] | 100% Completed |  0.1s


In [22]:
ff1_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff1_precip_lm_annual_aus)
ff2_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff2_precip_lm_annual_aus)
ff3_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff3_precip_lm_annual_aus)
ff4_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff4_precip_lm_annual_aus)
ff5_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff5_precip_lm_annual_aus)
ff6_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff6_precip_lm_annual_aus)
ff7_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff7_precip_lm_annual_aus)
ff8_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff8_precip_lm_annual_aus)
ff9_precip_lm_annual_southern_aus        = get_southern_aus_mean(ff9_precip_lm_annual_aus)
ff10_precip_lm_annual_southern_aus       = get_southern_aus_mean(ff10_precip_lm_annual_aus)
ff11_precip_lm_annual_southern_aus       = get_southern_aus_mean(ff11_precip_lm_annual_aus)
ff12_precip_lm_annual_southern_aus       = get_southern_aus_mean(ff12_precip_lm_annual_aus)
ff13_precip_lm_annual_southern_aus       = get_southern_aus_mean(ff13_precip_lm_annual_aus)
bcc_precip_lm_annual_southern_aus        = get_southern_aus_mean(bcc_precip_lm_annual_aus)
ccsm4_precip_lm_annual_southern_aus      = get_southern_aus_mean(ccsm4_precip_lm_annual_aus)
csiro_mk3l_precip_lm_annual_southern_aus = get_southern_aus_mean(csiro_mk3l_precip_lm_annual_aus)
fgoals_gl_precip_lm_annual_southern_aus  = get_southern_aus_mean(fgoals_gl_precip_lm_annual_aus)
fgoals_s2_precip_lm_annual_southern_aus  = get_southern_aus_mean(fgoals_s2_precip_lm_annual_aus)
giss_21_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_21_precip_lm_annual_aus)
giss_22_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_22_precip_lm_annual_aus)
giss_23_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_23_precip_lm_annual_aus)
giss_24_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_24_precip_lm_annual_aus)
giss_25_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_25_precip_lm_annual_aus)
giss_26_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_26_precip_lm_annual_aus)
giss_27_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_27_precip_lm_annual_aus)
giss_28_precip_lm_annual_southern_aus    = get_southern_aus_mean(giss_28_precip_lm_annual_aus)
hadcm3_precip_lm_annual_southern_aus     = get_southern_aus_mean(hadcm3_precip_lm_annual_aus)
ipsl_precip_lm_annual_southern_aus       = get_southern_aus_mean(ipsl_precip_lm_annual_aus)
miroc_precip_lm_annual_southern_aus      = get_southern_aus_mean(miroc_precip_lm_annual_aus)
mpi_precip_lm_annual_southern_aus        = get_southern_aus_mean(mpi_precip_lm_annual_aus)
mri_precip_lm_annual_southern_aus        = get_southern_aus_mean(mri_precip_lm_annual_aus)

### Recalculate drought metrics based on mean precip from the region

In [23]:
ff1_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff1_precip_hist_annual_southern_aus, 1900, 2000)    
ff2_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff2_precip_hist_annual_southern_aus, 1900, 2000)    
ff3_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff3_precip_hist_annual_southern_aus, 1900, 2000)    
ff4_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff4_precip_hist_annual_southern_aus, 1900, 2000)    
ff5_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff5_precip_hist_annual_southern_aus, 1900, 2000)    
ff6_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff6_precip_hist_annual_southern_aus, 1900, 2000)    
ff7_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff7_precip_hist_annual_southern_aus, 1900, 2000)    
ff8_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff8_precip_hist_annual_southern_aus, 1900, 2000)    
ff9_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff9_precip_hist_annual_southern_aus, 1900, 2000)    
ff10_precip_hist_annual_southern_aus_rc      = droughts_lastmillennium_fromyear(ff10_precip_hist_annual_southern_aus, 1900, 2000)    
ff11_precip_hist_annual_southern_aus_rc      = droughts_lastmillennium_fromyear(ff11_precip_hist_annual_southern_aus, 1900, 2000)    
ff12_precip_hist_annual_southern_aus_rc      = droughts_lastmillennium_fromyear(ff12_precip_hist_annual_southern_aus, 1900, 2000)    
ff13_precip_hist_annual_southern_aus_rc      = droughts_lastmillennium_fromyear(ff13_precip_hist_annual_southern_aus, 1900, 2000)    
bcc_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(bcc_precip_hist_annual_southern_aus, 1900, 2000)    
ccsm4_precip_hist_annual_southern_aus_rc     = droughts_lastmillennium_fromyear(ccsm4_precip_hist_annual_southern_aus     , 1900, 2000)
csiro_mk3l_precip_hist_annual_southern_aus_rc= droughts_lastmillennium_fromyear(csiro_mk3l_precip_hist_annual_southern_aus, 1900, 2000)
fgoals_gl_precip_hist_annual_southern_aus_rc = droughts_lastmillennium_fromyear(fgoals_gl_precip_hist_annual_southern_aus , 1900, 2000)
fgoals_s2_precip_hist_annual_southern_aus_rc = droughts_lastmillennium_fromyear(fgoals_s2_precip_hist_annual_southern_aus , 1900, 2000)
giss_21_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_21_precip_hist_annual_southern_aus   , 1900, 2000)
giss_22_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_22_precip_hist_annual_southern_aus   , 1900, 2000)
giss_23_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_23_precip_hist_annual_southern_aus   , 1900, 2000)
giss_24_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_24_precip_hist_annual_southern_aus   , 1900, 2000)
giss_25_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_25_precip_hist_annual_southern_aus   , 1900, 2000)
giss_26_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_26_precip_hist_annual_southern_aus   , 1900, 2000)
giss_27_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_27_precip_hist_annual_southern_aus   , 1900, 2000)
giss_28_precip_hist_annual_southern_aus_rc   = droughts_lastmillennium_fromyear(giss_28_precip_hist_annual_southern_aus   , 1900, 2000)
hadcm3_precip_hist_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(hadcm3_precip_hist_annual_southern_aus    , 1900, 2000)
ipsl_precip_hist_annual_southern_aus_rc      = droughts_lastmillennium_fromyear(ipsl_precip_hist_annual_southern_aus      , 1900, 2000)
miroc_precip_hist_annual_southern_aus_rc     = droughts_lastmillennium_fromyear(miroc_precip_hist_annual_southern_aus     , 1900, 2000)
mpi_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(mpi_precip_hist_annual_southern_aus       , 1900, 2000)
mri_precip_hist_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(mri_precip_hist_annual_southern_aus       , 1900, 2000)

In [12]:
awap_masked_annual_southern_aus_rc = droughts_lastmillennium_fromyear(awap_masked_annual_southern_aus, 1900, 2000)

In [25]:
ff_all_precip_hist_annual_southern_aus = xr.concat([ff1_precip_hist_annual_southern_aus, ff2_precip_hist_annual_southern_aus,
                                                    ff3_precip_hist_annual_southern_aus, ff4_precip_hist_annual_southern_aus, 
                                                    ff5_precip_hist_annual_southern_aus, ff6_precip_hist_annual_southern_aus, 
                                                    ff7_precip_hist_annual_southern_aus, ff8_precip_hist_annual_southern_aus, 
                                                    ff9_precip_hist_annual_southern_aus, ff10_precip_hist_annual_southern_aus,
                                                    ff11_precip_hist_annual_southern_aus, ff12_precip_hist_annual_southern_aus,
                                                    ff13_precip_hist_annual_southern_aus], dim='en')
ff_all_precip_hist_annual_southern_aus_mean = ff_all_precip_hist_annual_southern_aus.mean(dim='en')
ff_all_precip_hist_annual_southern_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_hist_annual_southern_aus_mean, 1900,2000)

giss_all_precip_hist_annual_southern_aus = xr.concat([giss_21_precip_hist_annual_southern_aus,
                                                      giss_22_precip_hist_annual_southern_aus,
                                                      giss_23_precip_hist_annual_southern_aus,
                                                      giss_24_precip_hist_annual_southern_aus,
                                                      giss_25_precip_hist_annual_southern_aus,
                                                      giss_26_precip_hist_annual_southern_aus,
                                                      giss_27_precip_hist_annual_southern_aus,
                                                      giss_28_precip_hist_annual_southern_aus], dim='en')
giss_all_precip_hist_annual_southern_aus_mean = giss_all_precip_hist_annual_southern_aus.mean(dim='en')
giss_all_precip_hist_annual_southern_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_hist_annual_southern_aus_mean, 1900, 2000)



In [26]:
save_netcdf_compression(ff1_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff1_precip_hist_annual_southern_aus')
save_netcdf_compression(ff2_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff2_precip_hist_annual_southern_aus')
save_netcdf_compression(ff3_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff3_precip_hist_annual_southern_aus')
save_netcdf_compression(ff4_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff4_precip_hist_annual_southern_aus')
save_netcdf_compression(ff5_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff5_precip_hist_annual_southern_aus')
save_netcdf_compression(ff6_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff6_precip_hist_annual_southern_aus')
save_netcdf_compression(ff7_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff7_precip_hist_annual_southern_aus')
save_netcdf_compression(ff8_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff8_precip_hist_annual_southern_aus')
save_netcdf_compression(ff9_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff9_precip_hist_annual_southern_aus')
save_netcdf_compression(ff10_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff10_precip_hist_annual_southern_aus')
save_netcdf_compression(ff11_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff11_precip_hist_annual_southern_aus')
save_netcdf_compression(ff12_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff12_precip_hist_annual_southern_aus')
save_netcdf_compression(ff13_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ff13_precip_hist_annual_southern_aus')

save_netcdf_compression(bcc_precip_hist_annual_southern_aus_rc, input_dir_hist, 'bcc_precip_hist_annual_southern_aus')
save_netcdf_compression(ccsm4_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ccsm4_precip_hist_annual_southern_aus')
save_netcdf_compression(csiro_mk3l_precip_hist_annual_southern_aus_rc, input_dir_hist, 'csiro_mk3l_precip_hist_annual_southern_aus')
save_netcdf_compression(fgoals_gl_precip_hist_annual_southern_aus_rc, input_dir_hist, 'fgoals_gl_precip_hist_annual_southern_aus')
save_netcdf_compression(fgoals_s2_precip_hist_annual_southern_aus_rc, input_dir_hist, 'fgoals_s2_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_21_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_21_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_22_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_22_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_23_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_23_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_24_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_24_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_25_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_25_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_26_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_26_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_27_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_27_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_28_precip_hist_annual_southern_aus_rc, input_dir_hist, 'giss_28_precip_hist_annual_southern_aus')
save_netcdf_compression(hadcm3_precip_hist_annual_southern_aus_rc, input_dir_hist, 'hadcm3_precip_hist_annual_southern_aus')
save_netcdf_compression(ipsl_precip_hist_annual_southern_aus_rc, input_dir_hist, 'ipsl_precip_hist_annual_southern_aus')
save_netcdf_compression(miroc_precip_hist_annual_southern_aus_rc, input_dir_hist, 'miroc_precip_hist_annual_southern_aus')
save_netcdf_compression(mpi_precip_hist_annual_southern_aus_rc, input_dir_hist, 'mpi_precip_hist_annual_southern_aus')
save_netcdf_compression(mri_precip_hist_annual_southern_aus_rc, input_dir_hist, 'mri_precip_hist_annual_southern_aus')
save_netcdf_compression(ff_all_precip_hist_annual_southern_aus_mean_rc, input_dir_hist, 'ff_all_precip_hist_annual_southern_aus')
save_netcdf_compression(giss_all_precip_hist_annual_southern_aus_mean_rc, input_dir_hist, 'giss_all_precip_hist_annual_southern_aus')



[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [27]:
ff1_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff1_precip_lm_annual_southern_aus       , 1900, 2000)
ff2_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff2_precip_lm_annual_southern_aus       , 1900, 2000)
ff3_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff3_precip_lm_annual_southern_aus       , 1900, 2000)
ff4_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff4_precip_lm_annual_southern_aus       , 1900, 2000)
ff5_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff5_precip_lm_annual_southern_aus       , 1900, 2000)
ff6_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff6_precip_lm_annual_southern_aus       , 1900, 2000)
ff7_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff7_precip_lm_annual_southern_aus       , 1900, 2000)
ff8_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff8_precip_lm_annual_southern_aus       , 1900, 2000)
ff9_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(ff9_precip_lm_annual_southern_aus       , 1900, 2000)
ff10_precip_lm_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff10_precip_lm_annual_southern_aus      , 1900, 2000)
ff11_precip_lm_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff11_precip_lm_annual_southern_aus      , 1900, 2000)
ff12_precip_lm_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff12_precip_lm_annual_southern_aus      , 1900, 2000)
ff13_precip_lm_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ff13_precip_lm_annual_southern_aus      , 1900, 2000)
bcc_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(bcc_precip_lm_annual_southern_aus       , 1900, 2000)
ccsm4_precip_lm_annual_southern_aus_rc      = droughts_lastmillennium_fromyear(ccsm4_precip_lm_annual_southern_aus     , 1900, 2000)
csiro_mk3l_precip_lm_annual_southern_aus_rc = droughts_lastmillennium_fromyear(csiro_mk3l_precip_lm_annual_southern_aus, 1900, 2000)
fgoals_gl_precip_lm_annual_southern_aus_rc  = droughts_lastmillennium_fromyear(fgoals_gl_precip_lm_annual_southern_aus , 1900, 2000)
fgoals_s2_precip_lm_annual_southern_aus_rc  = droughts_lastmillennium_fromyear(fgoals_s2_precip_lm_annual_southern_aus , 1900, 2000)
giss_21_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_21_precip_lm_annual_southern_aus   , 1900, 2000)
giss_22_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_22_precip_lm_annual_southern_aus   , 1900, 2000)
giss_23_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_23_precip_lm_annual_southern_aus   , 1900, 2000)
giss_24_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_24_precip_lm_annual_southern_aus   , 1900, 2000)
giss_25_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_25_precip_lm_annual_southern_aus   , 1900, 2000)
giss_26_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_26_precip_lm_annual_southern_aus   , 1900, 2000)
giss_27_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_27_precip_lm_annual_southern_aus   , 1900, 2000)
giss_28_precip_lm_annual_southern_aus_rc    = droughts_lastmillennium_fromyear(giss_28_precip_lm_annual_southern_aus   , 1900, 2000)
hadcm3_precip_lm_annual_southern_aus_rc     = droughts_lastmillennium_fromyear(hadcm3_precip_lm_annual_southern_aus    , 1900, 2000)
ipsl_precip_lm_annual_southern_aus_rc       = droughts_lastmillennium_fromyear(ipsl_precip_lm_annual_southern_aus      , 1900, 2000)
miroc_precip_lm_annual_southern_aus_rc      = droughts_lastmillennium_fromyear(miroc_precip_lm_annual_southern_aus     , 1900, 2000)
mpi_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(mpi_precip_lm_annual_southern_aus       , 1900, 2000)
mri_precip_lm_annual_southern_aus_rc        = droughts_lastmillennium_fromyear(mri_precip_lm_annual_southern_aus       , 1900, 2000)

In [28]:
ff_all_precip_lm_annual_southern_aus = xr.concat([ff1_precip_lm_annual_southern_aus, ff2_precip_lm_annual_southern_aus,
                                                    ff3_precip_lm_annual_southern_aus, ff4_precip_lm_annual_southern_aus, 
                                                    ff5_precip_lm_annual_southern_aus, ff6_precip_lm_annual_southern_aus, 
                                                    ff7_precip_lm_annual_southern_aus, ff8_precip_lm_annual_southern_aus, 
                                                    ff9_precip_lm_annual_southern_aus, ff10_precip_lm_annual_southern_aus,
                                                    ff11_precip_lm_annual_southern_aus, ff12_precip_lm_annual_southern_aus,
                                                    ff13_precip_lm_annual_southern_aus], dim='en')
ff_all_precip_lm_annual_southern_aus_mean = ff_all_precip_lm_annual_southern_aus.mean(dim='en')
ff_all_precip_lm_annual_southern_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_lm_annual_southern_aus_mean, 1900,2000)

giss_all_precip_lm_annual_southern_aus = xr.concat([giss_21_precip_lm_annual_southern_aus,
                                                      giss_22_precip_lm_annual_southern_aus,
                                                      giss_23_precip_lm_annual_southern_aus,
                                                      giss_24_precip_lm_annual_southern_aus,
                                                      giss_25_precip_lm_annual_southern_aus,
                                                      giss_26_precip_lm_annual_southern_aus,
                                                      giss_27_precip_lm_annual_southern_aus,
                                                      giss_28_precip_lm_annual_southern_aus], dim='en')
giss_all_precip_lm_annual_southern_aus_mean = giss_all_precip_lm_annual_southern_aus.mean(dim='en')
giss_all_precip_lm_annual_southern_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_lm_annual_southern_aus_mean, 1900, 2000)



In [29]:
save_netcdf_compression(ff1_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff1_precip_lm_annual_southern_aus')
save_netcdf_compression(ff2_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff2_precip_lm_annual_southern_aus')
save_netcdf_compression(ff3_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff3_precip_lm_annual_southern_aus')
save_netcdf_compression(ff4_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff4_precip_lm_annual_southern_aus')
save_netcdf_compression(ff5_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff5_precip_lm_annual_southern_aus')
save_netcdf_compression(ff6_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff6_precip_lm_annual_southern_aus')
save_netcdf_compression(ff7_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff7_precip_lm_annual_southern_aus')
save_netcdf_compression(ff8_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff8_precip_lm_annual_southern_aus')
save_netcdf_compression(ff9_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff9_precip_lm_annual_southern_aus')
save_netcdf_compression(ff10_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff10_precip_lm_annual_southern_aus')
save_netcdf_compression(ff11_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff11_precip_lm_annual_southern_aus')
save_netcdf_compression(ff12_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff12_precip_lm_annual_southern_aus')
save_netcdf_compression(ff13_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ff13_precip_lm_annual_southern_aus')

save_netcdf_compression(bcc_precip_lm_annual_southern_aus_rc, input_dir_lm, 'bcc_precip_lm_annual_southern_aus')
save_netcdf_compression(ccsm4_precip_lm_annual_southern_aus_rc, input_dir_lm, 'ccsm4_precip_lm_annual_southern_aus')
save_netcdf_compression(csiro_mk3l_precip_lm_annual_southern_aus_rc, input_dir_lm, 'csiro_mk3l_precip_lm_annual_southern_aus')
save_netcdf_compression(fgoals_gl_precip_lm_annual_southern_aus_rc,  input_dir_lm, 'fgoals_gl_precip_lm_annual_southern_aus')
save_netcdf_compression(fgoals_s2_precip_lm_annual_southern_aus_rc,  input_dir_lm, 'fgoals_s2_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_21_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_21_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_22_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_22_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_23_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_23_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_24_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_24_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_25_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_25_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_26_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_26_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_27_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_27_precip_lm_annual_southern_aus')
save_netcdf_compression(giss_28_precip_lm_annual_southern_aus_rc, input_dir_lm, 'giss_28_precip_lm_annual_southern_aus')
save_netcdf_compression(hadcm3_precip_lm_annual_southern_aus_rc, input_dir_lm, 'hadcm3_precip_lm_annual_southern_aus')
save_netcdf_compression(ipsl_precip_lm_annual_southern_aus_rc,   input_dir_lm, 'ipsl_precip_lm_annual_southern_aus')
save_netcdf_compression(miroc_precip_lm_annual_southern_aus_rc,  input_dir_lm, 'miroc_precip_lm_annual_southern_aus')
save_netcdf_compression(mpi_precip_lm_annual_southern_aus_rc, input_dir_lm, 'mpi_precip_lm_annual_southern_aus')
save_netcdf_compression(mri_precip_lm_annual_southern_aus_rc, input_dir_lm, 'mri_precip_lm_annual_southern_aus')

save_netcdf_compression(giss_all_precip_lm_annual_southern_aus_mean_rc, input_dir_lm, 'giss_all_precip_lm_annual_southern_aus_rc')
save_netcdf_compression(ff_all_precip_lm_annual_southern_aus_mean_rc,   input_dir_lm, 'ff_all_precip_lm_annual_southern_aus_rc')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

### create ds of all of the precip averages

In [32]:
ds_southern_aus_prect = awap_masked_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect = ds_southern_aus_prect.to_dataset()
ds_southern_aus_prect['AWAP'] = ds_southern_aus_prect.PRECT_mm
ds_southern_aus_prect = ds_southern_aus_prect.drop(['PRECT_mm'])

ds_southern_aus_prect['bcc'] = bcc_precip_hist_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect['ccsm4'] = ccsm4_precip_hist_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect['csiro_mk3l'] = csiro_mk3l_precip_hist_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect['fgoals_gl'] = fgoals_gl_precip_hist_annual_southern_aus_rc.PRECT_mm 
ds_southern_aus_prect['fgoals_s2'] = fgoals_s2_precip_hist_annual_southern_aus_rc.PRECT_mm 
ds_southern_aus_prect['giss_21'] = giss_21_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['giss_22'] = giss_22_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['giss_23'] = giss_23_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['giss_24'] = giss_24_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['giss_25'] = giss_25_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['giss_26'] = giss_26_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['giss_27'] = giss_27_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['giss_28'] = giss_28_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['hadcm3'] = hadcm3_precip_hist_annual_southern_aus_rc.PRECT_mm    
ds_southern_aus_prect['ipsl'] = ipsl_precip_hist_annual_southern_aus_rc.PRECT_mm      
ds_southern_aus_prect['miroc'] = miroc_precip_hist_annual_southern_aus_rc.PRECT_mm     
ds_southern_aus_prect['mpi'] = mpi_precip_hist_annual_southern_aus_rc.PRECT_mm       
ds_southern_aus_prect['mri'] = mri_precip_hist_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect['ff1'] = ff1_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff2'] = ff2_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff3'] = ff3_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff4'] = ff4_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff5'] = ff5_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff6'] = ff6_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff7'] = ff7_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff8'] = ff8_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff9'] = ff9_precip_hist_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect['ff10'] = ff10_precip_hist_annual_southern_aus_rc.PRECT_mm  
ds_southern_aus_prect['ff11'] = ff11_precip_hist_annual_southern_aus_rc.PRECT_mm  
ds_southern_aus_prect['ff12'] = ff12_precip_hist_annual_southern_aus_rc.PRECT_mm  
ds_southern_aus_prect['ff13'] = ff13_precip_hist_annual_southern_aus_rc.PRECT_mm

save_netcdf_compression(ds_southern_aus_prect, input_dir_hist, 'pmip3_models_precip_hist_southern_aus')


[########################################] | 100% Completed |  0.1s


In [33]:
ds_southern_aus_prect_lm = bcc_precip_lm_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect_lm = ds_southern_aus_prect_lm.to_dataset()
ds_southern_aus_prect_lm['bcc'] = ds_southern_aus_prect_lm.PRECT_mm

ds_southern_aus_prect_lm['ccsm4'] = ccsm4_precip_lm_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect_lm['csiro_mk3l'] = csiro_mk3l_precip_lm_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect_lm['fgoals_gl'] = fgoals_gl_precip_lm_annual_southern_aus_rc.PRECT_mm 
ds_southern_aus_prect_lm['fgoals_s2'] = fgoals_s2_precip_lm_annual_southern_aus_rc.PRECT_mm 
ds_southern_aus_prect_lm['giss_21'] = giss_21_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['giss_22'] = giss_22_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['giss_23'] = giss_23_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['giss_24'] = giss_24_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['giss_25'] = giss_25_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['giss_26'] = giss_26_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['giss_27'] = giss_27_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['giss_28'] = giss_28_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['hadcm3'] = hadcm3_precip_lm_annual_southern_aus_rc.PRECT_mm    
ds_southern_aus_prect_lm['ipsl'] = ipsl_precip_lm_annual_southern_aus_rc.PRECT_mm      
ds_southern_aus_prect_lm['miroc'] = miroc_precip_lm_annual_southern_aus_rc.PRECT_mm     
ds_southern_aus_prect_lm['mpi'] = mpi_precip_lm_annual_southern_aus_rc.PRECT_mm       
ds_southern_aus_prect_lm['mri'] = mri_precip_lm_annual_southern_aus_rc.PRECT_mm
ds_southern_aus_prect_lm['ff1'] = ff1_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff2'] = ff2_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff3'] = ff3_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff4'] = ff4_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff5'] = ff5_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff6'] = ff6_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff7'] = ff7_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff8'] = ff8_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff9'] = ff9_precip_lm_annual_southern_aus_rc.PRECT_mm   
ds_southern_aus_prect_lm['ff10'] = ff10_precip_lm_annual_southern_aus_rc.PRECT_mm  
ds_southern_aus_prect_lm['ff11'] = ff11_precip_lm_annual_southern_aus_rc.PRECT_mm  
ds_southern_aus_prect_lm['ff12'] = ff12_precip_lm_annual_southern_aus_rc.PRECT_mm  
ds_southern_aus_prect_lm['ff13'] = ff13_precip_lm_annual_southern_aus_rc.PRECT_mm

ds_southern_aus_prect_lm = ds_southern_aus_prect_lm.drop(('PRECT_mm'))

save_netcdf_compression(ds_southern_aus_prect_lm, input_dir_lm, 'pmip3_models_precip_lm_southern_aus')

[########################################] | 100% Completed |  0.1s


# Bootstrap

In [35]:
bcc_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.bcc, 'bcc')
ccsm4_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ccsm4, 'ccsm4')
csiro_mk3l_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.csiro_mk3l, 'csiro_mk3l')
fgoals_gl_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.fgoals_gl, 'fgoals_gl')
fgoals_s2_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.fgoals_s2, 'fgoals_s2')
giss_21_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_21, 'giss_21')
giss_22_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_22, 'giss_22')
giss_23_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_23, 'giss_23')
giss_24_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_24, 'giss_24')
giss_25_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_25, 'giss_25')
giss_26_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_26, 'giss_26')
giss_27_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_27, 'giss_27')
giss_28_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.giss_28, 'giss_28')
hadcm3_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.hadcm3, 'hadcm3')
ipsl_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ipsl, 'ipsl')
miroc_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.miroc, 'miroc')
mpi_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.mpi, 'mpi')
mri_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.mri, 'mri')
ff1_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff1, 'ff1')
ff2_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff2, 'ff2')
ff3_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff3, 'ff3')
ff4_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff4, 'ff4')
ff5_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff5, 'ff5')
ff6_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff6, 'ff6')
ff7_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff7, 'ff7')
ff8_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff8, 'ff8')
ff9_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff9, 'ff9')
ff10_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff10, 'ff10')
ff11_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff11, 'ff11')
ff12_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff12, 'ff12')
ff13_boot_southern_aus = bootstrap_years_101(ds_southern_aus_prect_lm.ff13, 'ff13')

/Users/nickywright/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1370: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input, interpolation=interpolation
/Users/nickywright/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1370: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input, interpolation=interpolation
/Users/nickywright/miniconda3/lib/python3.7/site-packages/xarray/core/computation.py:700: RuntimeWarning: All-NaN slice encountered
  result_data = func(*input_data)
/Users/nickywright/miniconda3/lib/python3.7/site-packages/xarray/core/computation.py:700: RuntimeWarning: Mean of empty slice
  result_data = func(*input_data)
/Users/nickywright/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/nickywright/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1370: RuntimeWarning: All-NaN slice en

KeyboardInterrupt: 

In [ ]:
ff_all_precip_lm_annual_southern_aus_mean_rc_boots = bootstrap_years_101(ff_all_precip_lm_annual_southern_aus_mean_rc.PRECT_mm, 'PRECT_mm')
# ff_all_precip_lm_annual_southern_aus_mean_rc_boots.to_netcdf('proc_bootstrap_everyyr/ff_all_boot_southern_aus.nc')

giss_all_precip_lm_annual_southern_aus_mean_rc_boots = bootstrap_years_101(giss_all_precip_lm_annual_southern_aus_mean_rc.PRECT_mm, 'PRECT_mm')
# giss_all_precip_lm_annual_southern_aus_mean_rc_boots.to_netcdf('proc_bootstrap_everyyr/giss_all_boot_southern_aus.nc')

In [ ]:
bootstrap_dir = 'proc_bootstrap_everyyr'
save_netcdf_compression(bcc_boot_southern_aus, bootstrap_dir, 'bcc_boot_southern_aus')
save_netcdf_compression(ccsm4_boot_southern_aus, bootstrap_dir, 'ccsm4_boot_southern_aus')
save_netcdf_compression(csiro_mk3l_boot_southern_aus, bootstrap_dir, 'csiro_mk3l_boot_southern_aus')
save_netcdf_compression(fgoals_gl_boot_southern_aus, bootstrap_dir, 'fgoals_gl_boot_southern_aus')
save_netcdf_compression(fgoals_s2_boot_southern_aus, bootstrap_dir, 'fgoals_s2_boot_southern_aus')
save_netcdf_compression(giss_21_boot_southern_aus, bootstrap_dir, 'giss_21_boot_southern_aus')
save_netcdf_compression(giss_22_boot_southern_aus, bootstrap_dir, 'giss_22_boot_southern_aus')
save_netcdf_compression(giss_23_boot_southern_aus, bootstrap_dir, 'giss_23_boot_southern_aus')
save_netcdf_compression(giss_24_boot_southern_aus, bootstrap_dir, 'giss_24_boot_southern_aus')
save_netcdf_compression(giss_25_boot_southern_aus, bootstrap_dir, 'giss_25_boot_southern_aus')
save_netcdf_compression(giss_26_boot_southern_aus, bootstrap_dir, 'giss_26_boot_southern_aus')
save_netcdf_compression(giss_27_boot_southern_aus, bootstrap_dir, 'giss_27_boot_southern_aus')
save_netcdf_compression(giss_28_boot_southern_aus, bootstrap_dir, 'giss_28_boot_southern_aus')
save_netcdf_compression(hadcm3_boot_southern_aus, bootstrap_dir, 'hadcm3_boot_southern_aus')
save_netcdf_compression(ipsl_boot_southern_aus, bootstrap_dir, 'ipsl_boot_southern_aus')
save_netcdf_compression(miroc_boot_southern_aus, bootstrap_dir, 'miroc_boot_southern_aus')
save_netcdf_compression(mpi_boot_southern_aus, bootstrap_dir, 'mpi_boot_southern_aus')
save_netcdf_compression(mri_boot_southern_aus, bootstrap_dir, 'mri_boot_southern_aus')
save_netcdf_compression(ff1_boot_southern_aus, bootstrap_dir, 'ff1_boot_southern_aus')
save_netcdf_compression(ff2_boot_southern_aus, bootstrap_dir, 'ff2_boot_southern_aus')
save_netcdf_compression(ff3_boot_southern_aus, bootstrap_dir, 'ff3_boot_southern_aus')
save_netcdf_compression(ff4_boot_southern_aus, bootstrap_dir, 'ff4_boot_southern_aus')
save_netcdf_compression(ff5_boot_southern_aus, bootstrap_dir, 'ff5_boot_southern_aus')
save_netcdf_compression(ff6_boot_southern_aus, bootstrap_dir, 'ff6_boot_southern_aus')
save_netcdf_compression(ff7_boot_southern_aus, bootstrap_dir, 'ff7_boot_southern_aus')
save_netcdf_compression(ff8_boot_southern_aus, bootstrap_dir, 'ff8_boot_southern_aus')
save_netcdf_compression(ff9_boot_southern_aus, bootstrap_dir, 'ff9_boot_southern_aus')
save_netcdf_compression(ff10_boot_southern_aus, bootstrap_dir, 'ff10_boot_southern_aus')
save_netcdf_compression(ff11_boot_southern_aus, bootstrap_dir, 'ff11_boot_southern_aus')
save_netcdf_compression(ff12_boot_southern_aus, bootstrap_dir, 'ff12_boot_southern_aus')
save_netcdf_compression(ff13_boot_southern_aus, bootstrap_dir, 'ff13_boot_southern_aus')
save_netcdf_compression(ff_all_precip_lm_annual_southern_aus_mean_rc_boots, bootstrap_dir, 'ff_all_boot_southern_aus')
save_netcdf_compression(giss_all_precip_lm_annual_southern_aus_mean_rc_boots, bootstrap_dir, 'giss_all_boot_southern_aus')

## Northern Aus

In [36]:
def get_northern_aus_mean(ds):
    ds_northern = ds.where(ds.lat >= -26, drop=True)
    ds_northern_mean = climate_xr_funcs.weighted_mean_CAM(ds_northern)
    return ds_northern_mean

In [37]:
ff1_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff1_precip_hist_annual_aus)
ff2_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff2_precip_hist_annual_aus)
ff3_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff3_precip_hist_annual_aus)
ff4_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff4_precip_hist_annual_aus)
ff5_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff5_precip_hist_annual_aus)
ff6_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff6_precip_hist_annual_aus)
ff7_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff7_precip_hist_annual_aus)
ff8_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff8_precip_hist_annual_aus)
ff9_precip_hist_annual_northern_aus        = get_northern_aus_mean(ff9_precip_hist_annual_aus)
ff10_precip_hist_annual_northern_aus       = get_northern_aus_mean(ff10_precip_hist_annual_aus)
ff11_precip_hist_annual_northern_aus       = get_northern_aus_mean(ff11_precip_hist_annual_aus)
ff12_precip_hist_annual_northern_aus       = get_northern_aus_mean(ff12_precip_hist_annual_aus)
ff13_precip_hist_annual_northern_aus       = get_northern_aus_mean(ff13_precip_hist_annual_aus)

bcc_precip_hist_annual_northern_aus        = get_northern_aus_mean(bcc_precip_hist_annual_aus)
ccsm4_precip_hist_annual_northern_aus      = get_northern_aus_mean(ccsm4_precip_hist_annual_aus)
csiro_mk3l_precip_hist_annual_northern_aus = get_northern_aus_mean(csiro_mk3l_precip_hist_annual_aus)
fgoals_gl_precip_hist_annual_northern_aus  = get_northern_aus_mean(fgoals_gl_precip_hist_annual_aus)
fgoals_s2_precip_hist_annual_northern_aus  = get_northern_aus_mean(fgoals_s2_precip_hist_annual_aus)
giss_21_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_21_precip_hist_annual_aus)
giss_22_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_22_precip_hist_annual_aus)
giss_23_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_23_precip_hist_annual_aus)
giss_24_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_24_precip_hist_annual_aus)
giss_25_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_25_precip_hist_annual_aus)
giss_26_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_26_precip_hist_annual_aus)
giss_27_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_27_precip_hist_annual_aus)
giss_28_precip_hist_annual_northern_aus    = get_northern_aus_mean(giss_28_precip_hist_annual_aus)
hadcm3_precip_hist_annual_northern_aus     = get_northern_aus_mean(hadcm3_precip_hist_annual_aus)
ipsl_precip_hist_annual_northern_aus       = get_northern_aus_mean(ipsl_precip_hist_annual_aus)
miroc_precip_hist_annual_northern_aus      = get_northern_aus_mean(miroc_precip_hist_annual_aus)
mpi_precip_hist_annual_northern_aus        = get_northern_aus_mean(mpi_precip_hist_annual_aus)
mri_precip_hist_annual_northern_aus        = get_northern_aus_mean(mri_precip_hist_annual_aus)

ff1_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff1_precip_lm_annual_aus)
ff2_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff2_precip_lm_annual_aus)
ff3_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff3_precip_lm_annual_aus)
ff4_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff4_precip_lm_annual_aus)
ff5_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff5_precip_lm_annual_aus)
ff6_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff6_precip_lm_annual_aus)
ff7_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff7_precip_lm_annual_aus)
ff8_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff8_precip_lm_annual_aus)
ff9_precip_lm_annual_northern_aus        = get_northern_aus_mean(ff9_precip_lm_annual_aus)
ff10_precip_lm_annual_northern_aus       = get_northern_aus_mean(ff10_precip_lm_annual_aus)
ff11_precip_lm_annual_northern_aus       = get_northern_aus_mean(ff11_precip_lm_annual_aus)
ff12_precip_lm_annual_northern_aus       = get_northern_aus_mean(ff12_precip_lm_annual_aus)
ff13_precip_lm_annual_northern_aus       = get_northern_aus_mean(ff13_precip_lm_annual_aus)
bcc_precip_lm_annual_northern_aus        = get_northern_aus_mean(bcc_precip_lm_annual_aus)
ccsm4_precip_lm_annual_northern_aus      = get_northern_aus_mean(ccsm4_precip_lm_annual_aus)
csiro_mk3l_precip_lm_annual_northern_aus = get_northern_aus_mean(csiro_mk3l_precip_lm_annual_aus)
fgoals_gl_precip_lm_annual_northern_aus  = get_northern_aus_mean(fgoals_gl_precip_lm_annual_aus)
fgoals_s2_precip_lm_annual_northern_aus  = get_northern_aus_mean(fgoals_s2_precip_lm_annual_aus)
giss_21_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_21_precip_lm_annual_aus)
giss_22_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_22_precip_lm_annual_aus)
giss_23_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_23_precip_lm_annual_aus)
giss_24_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_24_precip_lm_annual_aus)
giss_25_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_25_precip_lm_annual_aus)
giss_26_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_26_precip_lm_annual_aus)
giss_27_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_27_precip_lm_annual_aus)
giss_28_precip_lm_annual_northern_aus    = get_northern_aus_mean(giss_28_precip_lm_annual_aus)
hadcm3_precip_lm_annual_northern_aus     = get_northern_aus_mean(hadcm3_precip_lm_annual_aus)
ipsl_precip_lm_annual_northern_aus       = get_northern_aus_mean(ipsl_precip_lm_annual_aus)
miroc_precip_lm_annual_northern_aus      = get_northern_aus_mean(miroc_precip_lm_annual_aus)
mpi_precip_lm_annual_northern_aus        = get_northern_aus_mean(mpi_precip_lm_annual_aus)
mri_precip_lm_annual_northern_aus        = get_northern_aus_mean(mri_precip_lm_annual_aus)

awap_masked_annual_northern_aus = get_northern_aus_mean(awap_gf_annual)

In [38]:
ff1_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff1_precip_hist_annual_northern_aus, 1900, 2000)    
ff2_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff2_precip_hist_annual_northern_aus, 1900, 2000)    
ff3_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff3_precip_hist_annual_northern_aus, 1900, 2000)    
ff4_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff4_precip_hist_annual_northern_aus, 1900, 2000)    
ff5_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff5_precip_hist_annual_northern_aus, 1900, 2000)    
ff6_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff6_precip_hist_annual_northern_aus, 1900, 2000)    
ff7_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff7_precip_hist_annual_northern_aus, 1900, 2000)    
ff8_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff8_precip_hist_annual_northern_aus, 1900, 2000)    
ff9_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff9_precip_hist_annual_northern_aus, 1900, 2000)    
ff10_precip_hist_annual_northern_aus_rc      = droughts_lastmillennium_fromyear(ff10_precip_hist_annual_northern_aus, 1900, 2000)    
ff11_precip_hist_annual_northern_aus_rc      = droughts_lastmillennium_fromyear(ff11_precip_hist_annual_northern_aus, 1900, 2000)    
ff12_precip_hist_annual_northern_aus_rc      = droughts_lastmillennium_fromyear(ff12_precip_hist_annual_northern_aus, 1900, 2000)    
ff13_precip_hist_annual_northern_aus_rc      = droughts_lastmillennium_fromyear(ff13_precip_hist_annual_northern_aus, 1900, 2000)    
bcc_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(bcc_precip_hist_annual_northern_aus, 1900, 2000)    
ccsm4_precip_hist_annual_northern_aus_rc     = droughts_lastmillennium_fromyear(ccsm4_precip_hist_annual_northern_aus     , 1900, 2000)
csiro_mk3l_precip_hist_annual_northern_aus_rc= droughts_lastmillennium_fromyear(csiro_mk3l_precip_hist_annual_northern_aus, 1900, 2000)
fgoals_gl_precip_hist_annual_northern_aus_rc = droughts_lastmillennium_fromyear(fgoals_gl_precip_hist_annual_northern_aus , 1900, 2000)
fgoals_s2_precip_hist_annual_northern_aus_rc = droughts_lastmillennium_fromyear(fgoals_s2_precip_hist_annual_northern_aus , 1900, 2000)
giss_21_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_21_precip_hist_annual_northern_aus   , 1900, 2000)
giss_22_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_22_precip_hist_annual_northern_aus   , 1900, 2000)
giss_23_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_23_precip_hist_annual_northern_aus   , 1900, 2000)
giss_24_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_24_precip_hist_annual_northern_aus   , 1900, 2000)
giss_25_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_25_precip_hist_annual_northern_aus   , 1900, 2000)
giss_26_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_26_precip_hist_annual_northern_aus   , 1900, 2000)
giss_27_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_27_precip_hist_annual_northern_aus   , 1900, 2000)
giss_28_precip_hist_annual_northern_aus_rc   = droughts_lastmillennium_fromyear(giss_28_precip_hist_annual_northern_aus   , 1900, 2000)
hadcm3_precip_hist_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(hadcm3_precip_hist_annual_northern_aus    , 1900, 2000)
ipsl_precip_hist_annual_northern_aus_rc      = droughts_lastmillennium_fromyear(ipsl_precip_hist_annual_northern_aus      , 1900, 2000)
miroc_precip_hist_annual_northern_aus_rc     = droughts_lastmillennium_fromyear(miroc_precip_hist_annual_northern_aus     , 1900, 2000)
mpi_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(mpi_precip_hist_annual_northern_aus       , 1900, 2000)
mri_precip_hist_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(mri_precip_hist_annual_northern_aus       , 1900, 2000)

ff1_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff1_precip_lm_annual_northern_aus       , 1900, 2000)
ff2_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff2_precip_lm_annual_northern_aus       , 1900, 2000)
ff3_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff3_precip_lm_annual_northern_aus       , 1900, 2000)
ff4_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff4_precip_lm_annual_northern_aus       , 1900, 2000)
ff5_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff5_precip_lm_annual_northern_aus       , 1900, 2000)
ff6_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff6_precip_lm_annual_northern_aus       , 1900, 2000)
ff7_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff7_precip_lm_annual_northern_aus       , 1900, 2000)
ff8_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff8_precip_lm_annual_northern_aus       , 1900, 2000)
ff9_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(ff9_precip_lm_annual_northern_aus       , 1900, 2000)
ff10_precip_lm_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff10_precip_lm_annual_northern_aus      , 1900, 2000)
ff11_precip_lm_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff11_precip_lm_annual_northern_aus      , 1900, 2000)
ff12_precip_lm_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff12_precip_lm_annual_northern_aus      , 1900, 2000)
ff13_precip_lm_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ff13_precip_lm_annual_northern_aus      , 1900, 2000)
bcc_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(bcc_precip_lm_annual_northern_aus       , 1900, 2000)
ccsm4_precip_lm_annual_northern_aus_rc      = droughts_lastmillennium_fromyear(ccsm4_precip_lm_annual_northern_aus     , 1900, 2000)
csiro_mk3l_precip_lm_annual_northern_aus_rc = droughts_lastmillennium_fromyear(csiro_mk3l_precip_lm_annual_northern_aus, 1900, 2000)
fgoals_gl_precip_lm_annual_northern_aus_rc  = droughts_lastmillennium_fromyear(fgoals_gl_precip_lm_annual_northern_aus , 1900, 2000)
fgoals_s2_precip_lm_annual_northern_aus_rc  = droughts_lastmillennium_fromyear(fgoals_s2_precip_lm_annual_northern_aus , 1900, 2000)
giss_21_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_21_precip_lm_annual_northern_aus   , 1900, 2000)
giss_22_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_22_precip_lm_annual_northern_aus   , 1900, 2000)
giss_23_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_23_precip_lm_annual_northern_aus   , 1900, 2000)
giss_24_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_24_precip_lm_annual_northern_aus   , 1900, 2000)
giss_25_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_25_precip_lm_annual_northern_aus   , 1900, 2000)
giss_26_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_26_precip_lm_annual_northern_aus   , 1900, 2000)
giss_27_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_27_precip_lm_annual_northern_aus   , 1900, 2000)
giss_28_precip_lm_annual_northern_aus_rc    = droughts_lastmillennium_fromyear(giss_28_precip_lm_annual_northern_aus   , 1900, 2000)
hadcm3_precip_lm_annual_northern_aus_rc     = droughts_lastmillennium_fromyear(hadcm3_precip_lm_annual_northern_aus    , 1900, 2000)
ipsl_precip_lm_annual_northern_aus_rc       = droughts_lastmillennium_fromyear(ipsl_precip_lm_annual_northern_aus      , 1900, 2000)
miroc_precip_lm_annual_northern_aus_rc      = droughts_lastmillennium_fromyear(miroc_precip_lm_annual_northern_aus     , 1900, 2000)
mpi_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(mpi_precip_lm_annual_northern_aus       , 1900, 2000)
mri_precip_lm_annual_northern_aus_rc        = droughts_lastmillennium_fromyear(mri_precip_lm_annual_northern_aus       , 1900, 2000)

awap_masked_annual_northern_aus_rc = droughts_lastmillennium_fromyear(awap_masked_annual_northern_aus, 1900, 2000)

ff_all_precip_hist_annual_northern_aus = xr.concat([ff1_precip_hist_annual_northern_aus, ff2_precip_hist_annual_northern_aus,
                                                    ff3_precip_hist_annual_northern_aus, ff4_precip_hist_annual_northern_aus, 
                                                    ff5_precip_hist_annual_northern_aus, ff6_precip_hist_annual_northern_aus, 
                                                    ff7_precip_hist_annual_northern_aus, ff8_precip_hist_annual_northern_aus, 
                                                    ff9_precip_hist_annual_northern_aus, ff10_precip_hist_annual_northern_aus,
                                                    ff11_precip_hist_annual_northern_aus, ff12_precip_hist_annual_northern_aus,
                                                    ff13_precip_hist_annual_northern_aus], dim='en')
ff_all_precip_hist_annual_northern_aus_mean = ff_all_precip_hist_annual_northern_aus.mean(dim='en')
ff_all_precip_hist_annual_northern_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_hist_annual_northern_aus_mean, 1900,2000)

giss_all_precip_hist_annual_northern_aus = xr.concat([giss_21_precip_hist_annual_northern_aus,
                                                      giss_22_precip_hist_annual_northern_aus,
                                                      giss_23_precip_hist_annual_northern_aus,
                                                      giss_24_precip_hist_annual_northern_aus,
                                                      giss_25_precip_hist_annual_northern_aus,
                                                      giss_26_precip_hist_annual_northern_aus,
                                                      giss_27_precip_hist_annual_northern_aus,
                                                      giss_28_precip_hist_annual_northern_aus], dim='en')
giss_all_precip_hist_annual_northern_aus_mean = giss_all_precip_hist_annual_northern_aus.mean(dim='en')
giss_all_precip_hist_annual_northern_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_hist_annual_northern_aus_mean, 1900, 2000)

ff_all_precip_lm_annual_northern_aus = xr.concat([ff1_precip_lm_annual_northern_aus, ff2_precip_lm_annual_northern_aus,
                                                  ff3_precip_lm_annual_northern_aus, ff4_precip_lm_annual_northern_aus, 
                                                  ff5_precip_lm_annual_northern_aus, ff6_precip_lm_annual_northern_aus, 
                                                  ff7_precip_lm_annual_northern_aus, ff8_precip_lm_annual_northern_aus, 
                                                  ff9_precip_lm_annual_northern_aus, ff10_precip_lm_annual_northern_aus,
                                                  ff11_precip_lm_annual_northern_aus, ff12_precip_lm_annual_northern_aus,
                                                  ff13_precip_lm_annual_northern_aus], dim='en')
ff_all_precip_lm_annual_northern_aus_mean = ff_all_precip_lm_annual_northern_aus.mean(dim='en')
ff_all_precip_lm_annual_northern_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_lm_annual_northern_aus_mean, 1900,2000)

giss_all_precip_lm_annual_northern_aus = xr.concat([giss_21_precip_lm_annual_northern_aus,
                                                    giss_22_precip_lm_annual_northern_aus,
                                                    giss_23_precip_lm_annual_northern_aus,
                                                    giss_24_precip_lm_annual_northern_aus,
                                                    giss_25_precip_lm_annual_northern_aus,
                                                    giss_26_precip_lm_annual_northern_aus,
                                                    giss_27_precip_lm_annual_northern_aus,
                                                    giss_28_precip_lm_annual_northern_aus], dim='en')
giss_all_precip_lm_annual_northern_aus_mean = giss_all_precip_lm_annual_northern_aus.mean(dim='en')
giss_all_precip_lm_annual_northern_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_lm_annual_northern_aus_mean, 1900, 2000)



In [39]:
awap_masked_annual_northern_aus = get_northern_aus_mean(awap_gf_annual)
awap_masked_annual_northern_aus_rc = droughts_lastmillennium_fromyear(awap_masked_annual_northern_aus, 1900, 2000)
save_netcdf_compression(awap_masked_annual_northern_aus_rc, input_dir_hist, 'awap_masked_annual_northern_aus')



[########################################] | 100% Completed |  0.1s


### save file

In [40]:
save_netcdf_compression(ff1_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff1_precip_hist_annual_northern_aus')
save_netcdf_compression(ff2_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff2_precip_hist_annual_northern_aus')
save_netcdf_compression(ff3_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff3_precip_hist_annual_northern_aus')
save_netcdf_compression(ff4_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff4_precip_hist_annual_northern_aus')
save_netcdf_compression(ff5_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff5_precip_hist_annual_northern_aus')
save_netcdf_compression(ff6_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff6_precip_hist_annual_northern_aus')
save_netcdf_compression(ff7_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff7_precip_hist_annual_northern_aus')
save_netcdf_compression(ff8_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff8_precip_hist_annual_northern_aus')
save_netcdf_compression(ff9_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff9_precip_hist_annual_northern_aus')
save_netcdf_compression(ff10_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff10_precip_hist_annual_northern_aus')
save_netcdf_compression(ff11_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff11_precip_hist_annual_northern_aus')
save_netcdf_compression(ff12_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff12_precip_hist_annual_northern_aus')
save_netcdf_compression(ff13_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ff13_precip_hist_annual_northern_aus')

save_netcdf_compression(bcc_precip_hist_annual_northern_aus_rc, input_dir_hist, 'bcc_precip_hist_annual_northern_aus')
save_netcdf_compression(ccsm4_precip_hist_annual_northern_aus_rc, input_dir_hist, 'ccsm4_precip_hist_annual_northern_aus')
save_netcdf_compression(csiro_mk3l_precip_hist_annual_northern_aus_rc, input_dir_hist, 'csiro_mk3l_precip_hist_annual_northern_aus')
save_netcdf_compression(fgoals_gl_precip_hist_annual_northern_aus_rc,  input_dir_hist, 'fgoals_gl_precip_hist_annual_northern_aus')
save_netcdf_compression(fgoals_s2_precip_hist_annual_northern_aus_rc,  input_dir_hist, 'fgoals_s2_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_21_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_21_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_22_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_22_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_23_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_23_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_24_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_24_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_25_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_25_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_26_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_26_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_27_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_27_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_28_precip_hist_annual_northern_aus_rc, input_dir_hist, 'giss_28_precip_hist_annual_northern_aus')
save_netcdf_compression(hadcm3_precip_hist_annual_northern_aus_rc, input_dir_hist, 'hadcm3_precip_hist_annual_northern_aus')
save_netcdf_compression(ipsl_precip_hist_annual_northern_aus_rc,   input_dir_hist, 'ipsl_precip_hist_annual_northern_aus')
save_netcdf_compression(miroc_precip_hist_annual_northern_aus_rc, input_dir_hist, 'miroc_precip_hist_annual_northern_aus')
save_netcdf_compression(mpi_precip_hist_annual_northern_aus_rc,   input_dir_hist, 'mpi_precip_hist_annual_northern_aus')
save_netcdf_compression(mri_precip_hist_annual_northern_aus_rc,   input_dir_hist, 'mri_precip_hist_annual_northern_aus')

save_netcdf_compression(ff_all_precip_hist_annual_northern_aus_mean_rc, input_dir_hist, 'ff_all_precip_hist_annual_northern_aus')
save_netcdf_compression(giss_all_precip_hist_annual_northern_aus_mean_rc, input_dir_hist, 'giss_all_precip_hist_annual_northern_aus')


[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [42]:
save_netcdf_compression(ff1_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff1_precip_lm_annual_northern_aus')
save_netcdf_compression(ff2_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff2_precip_lm_annual_northern_aus')
save_netcdf_compression(ff3_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff3_precip_lm_annual_northern_aus')
save_netcdf_compression(ff4_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff4_precip_lm_annual_northern_aus')
save_netcdf_compression(ff5_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff5_precip_lm_annual_northern_aus')
save_netcdf_compression(ff6_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff6_precip_lm_annual_northern_aus')
save_netcdf_compression(ff7_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff7_precip_lm_annual_northern_aus')
save_netcdf_compression(ff8_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff8_precip_lm_annual_northern_aus')
save_netcdf_compression(ff9_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff9_precip_lm_annual_northern_aus')
save_netcdf_compression(ff10_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff10_precip_lm_annual_northern_aus')
save_netcdf_compression(ff11_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff11_precip_lm_annual_northern_aus')
save_netcdf_compression(ff12_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff12_precip_lm_annual_northern_aus')
save_netcdf_compression(ff13_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ff13_precip_lm_annual_northern_aus')

save_netcdf_compression(bcc_precip_lm_annual_northern_aus_rc,   input_dir_lm, 'bcc_precip_lm_annual_northern_aus')
save_netcdf_compression(ccsm4_precip_lm_annual_northern_aus_rc, input_dir_lm, 'ccsm4_precip_lm_annual_northern_aus')
save_netcdf_compression(csiro_mk3l_precip_lm_annual_northern_aus_rc, input_dir_lm, 'csiro_mk3l_precip_lm_annual_northern_aus')
save_netcdf_compression(fgoals_gl_precip_lm_annual_northern_aus_rc,  input_dir_lm, 'fgoals_gl_precip_lm_annual_northern_aus')
save_netcdf_compression(fgoals_s2_precip_lm_annual_northern_aus_rc,  input_dir_lm, 'fgoals_s2_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_21_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_21_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_22_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_22_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_23_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_23_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_24_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_24_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_25_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_25_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_26_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_26_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_27_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_27_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_28_precip_lm_annual_northern_aus_rc, input_dir_lm, 'giss_28_precip_lm_annual_northern_aus')
save_netcdf_compression(hadcm3_precip_lm_annual_northern_aus_rc, input_dir_lm, 'hadcm3_precip_lm_annual_northern_aus')
save_netcdf_compression(ipsl_precip_lm_annual_northern_aus_rc,   input_dir_lm, 'ipsl_precip_lm_annual_northern_aus')
save_netcdf_compression(miroc_precip_lm_annual_northern_aus_rc,  input_dir_lm, 'miroc_precip_lm_annual_northern_aus')
save_netcdf_compression(mpi_precip_lm_annual_northern_aus_rc, input_dir_lm, 'mpi_precip_lm_annual_northern_aus')
save_netcdf_compression(mri_precip_lm_annual_northern_aus_rc, input_dir_lm, 'mri_precip_lm_annual_northern_aus')
save_netcdf_compression(giss_all_precip_lm_annual_northern_aus_mean_rc, input_dir_lm, 'giss_all_precip_lm_annual_northern_aus')
save_netcdf_compression(ff_all_precip_lm_annual_northern_aus_mean_rc,   input_dir_lm, 'ff_all_precip_lm_annual_northern_aus')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [43]:
ds_northern_aus_prect = awap_masked_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect = ds_northern_aus_prect.to_dataset()
ds_northern_aus_prect['AWAP'] = ds_northern_aus_prect.PRECT_mm
ds_northern_aus_prect = ds_northern_aus_prect.drop(['PRECT_mm'])

ds_northern_aus_prect['bcc'] = bcc_precip_hist_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect['ccsm4'] = ccsm4_precip_hist_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect['csiro_mk3l'] = csiro_mk3l_precip_hist_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect['fgoals_gl'] = fgoals_gl_precip_hist_annual_northern_aus_rc.PRECT_mm 
ds_northern_aus_prect['fgoals_s2'] = fgoals_s2_precip_hist_annual_northern_aus_rc.PRECT_mm 
ds_northern_aus_prect['giss_21'] = giss_21_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['giss_22'] = giss_22_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['giss_23'] = giss_23_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['giss_24'] = giss_24_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['giss_25'] = giss_25_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['giss_26'] = giss_26_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['giss_27'] = giss_27_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['giss_28'] = giss_28_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['hadcm3'] = hadcm3_precip_hist_annual_northern_aus_rc.PRECT_mm    
ds_northern_aus_prect['ipsl'] = ipsl_precip_hist_annual_northern_aus_rc.PRECT_mm      
ds_northern_aus_prect['miroc'] = miroc_precip_hist_annual_northern_aus_rc.PRECT_mm     
ds_northern_aus_prect['mpi'] = mpi_precip_hist_annual_northern_aus_rc.PRECT_mm       
ds_northern_aus_prect['mri'] = mri_precip_hist_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect['ff1'] = ff1_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff2'] = ff2_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff3'] = ff3_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff4'] = ff4_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff5'] = ff5_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff6'] = ff6_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff7'] = ff7_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff8'] = ff8_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff9'] = ff9_precip_hist_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect['ff10'] = ff10_precip_hist_annual_northern_aus_rc.PRECT_mm  
ds_northern_aus_prect['ff11'] = ff11_precip_hist_annual_northern_aus_rc.PRECT_mm  
ds_northern_aus_prect['ff12'] = ff12_precip_hist_annual_northern_aus_rc.PRECT_mm  
ds_northern_aus_prect['ff13'] = ff13_precip_hist_annual_northern_aus_rc.PRECT_mm

# ds_northern_aus_prect = ds_northern_aus_prect.drop('quantile')
save_netcdf_compression(ds_northern_aus_prect, input_dir_hist, 'pmip3_models_precip_hist_northern_aus')



[########################################] | 100% Completed |  0.1s


In [45]:
# for the last millennium
ds_northern_aus_prect_lm = bcc_precip_lm_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect_lm = ds_northern_aus_prect_lm.to_dataset()
ds_northern_aus_prect_lm['bcc'] = ds_northern_aus_prect_lm.PRECT_mm

ds_northern_aus_prect_lm['ccsm4'] = ccsm4_precip_lm_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect_lm['csiro_mk3l'] = csiro_mk3l_precip_lm_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect_lm['fgoals_gl'] = fgoals_gl_precip_lm_annual_northern_aus_rc.PRECT_mm 
ds_northern_aus_prect_lm['fgoals_s2'] = fgoals_s2_precip_lm_annual_northern_aus_rc.PRECT_mm 
ds_northern_aus_prect_lm['giss_21'] = giss_21_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['giss_22'] = giss_22_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['giss_23'] = giss_23_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['giss_24'] = giss_24_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['giss_25'] = giss_25_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['giss_26'] = giss_26_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['giss_27'] = giss_27_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['giss_28'] = giss_28_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['hadcm3'] = hadcm3_precip_lm_annual_northern_aus_rc.PRECT_mm    
ds_northern_aus_prect_lm['ipsl'] = ipsl_precip_lm_annual_northern_aus_rc.PRECT_mm      
ds_northern_aus_prect_lm['miroc'] = miroc_precip_lm_annual_northern_aus_rc.PRECT_mm     
ds_northern_aus_prect_lm['mpi'] = mpi_precip_lm_annual_northern_aus_rc.PRECT_mm       
ds_northern_aus_prect_lm['mri'] = mri_precip_lm_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect_lm['ff1'] = ff1_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff2'] = ff2_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff3'] = ff3_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff4'] = ff4_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff5'] = ff5_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff6'] = ff6_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff7'] = ff7_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff8'] = ff8_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff9'] = ff9_precip_lm_annual_northern_aus_rc.PRECT_mm   
ds_northern_aus_prect_lm['ff10'] = ff10_precip_lm_annual_northern_aus_rc.PRECT_mm  
ds_northern_aus_prect_lm['ff11'] = ff11_precip_lm_annual_northern_aus_rc.PRECT_mm  
ds_northern_aus_prect_lm['ff12'] = ff12_precip_lm_annual_northern_aus_rc.PRECT_mm  
ds_northern_aus_prect_lm['ff13'] = ff13_precip_lm_annual_northern_aus_rc.PRECT_mm
ds_northern_aus_prect_lm = ds_northern_aus_prect_lm.drop(('PRECT_mm'))
save_netcdf_compression(ds_northern_aus_prect_lm, input_dir_lm, 'pmip3_models_precip_lm_northern_aus')

[########################################] | 100% Completed |  0.1s


### bootstrap

In [44]:
bcc_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.bcc, 'bcc')
ccsm4_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ccsm4, 'ccsm4')
csiro_mk3l_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.csiro_mk3l, 'csiro_mk3l')
fgoals_gl_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.fgoals_gl, 'fgoals_gl')
fgoals_s2_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.fgoals_s2, 'fgoals_s2')
giss_21_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_21, 'giss_21')
giss_22_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_22, 'giss_22')
giss_23_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_23, 'giss_23')
giss_24_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_24, 'giss_24')
giss_25_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_25, 'giss_25')
giss_26_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_26, 'giss_26')
giss_27_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_27, 'giss_27')
giss_28_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.giss_28, 'giss_28')
hadcm3_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.hadcm3, 'hadcm3')
ipsl_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ipsl, 'ipsl')
miroc_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.miroc, 'miroc')
mpi_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.mpi, 'mpi')
mri_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.mri, 'mri')
ff1_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff1, 'ff1')
ff2_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff2, 'ff2')
ff3_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff3, 'ff3')
ff4_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff4, 'ff4')
ff5_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff5, 'ff5')
ff6_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff6, 'ff6')
ff7_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff7, 'ff7')
ff8_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff8, 'ff8')
ff9_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff9, 'ff9')
ff10_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff10, 'ff10')
ff11_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff11, 'ff11')
ff12_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff12, 'ff12')
ff13_boot_northern_aus = bootstrap_years_101(ds_northern_aus_prect_lm.ff13, 'ff13')
ff_all_precip_lm_annual_northern_aus_mean_rc_boots = bootstrap_years_101(ff_all_precip_lm_annual_northern_aus_mean_rc.PRECT_mm, 'PRECT_mm')
giss_all_precip_lm_annual_northern_aus_mean_rc_boots = bootstrap_years_101(giss_all_precip_lm_annual_northern_aus_mean_rc.PRECT_mm, 'PRECT_mm')

NameError: name 'ds_northern_aus_prect_lm' is not defined

In [ ]:
save_netcdf_compression(bcc_boot_northern_aus, bootstrap_dir, 'bcc_boot_northern_aus')
save_netcdf_compression(ccsm4_boot_northern_aus, bootstrap_dir, 'ccsm4_boot_northern_aus')
save_netcdf_compression(csiro_mk3l_boot_northern_aus, bootstrap_dir, 'csiro_mk3l_boot_northern_aus')
save_netcdf_compression(fgoals_gl_boot_northern_aus,  bootstrap_dir, 'fgoals_gl_boot_northern_aus')
save_netcdf_compression(fgoals_s2_boot_northern_aus,  bootstrap_dir, 'fgoals_s2_boot_northern_aus')
save_netcdf_compression(giss_21_boot_northern_aus, bootstrap_dir, 'giss_21_boot_northern_aus')
save_netcdf_compression(giss_22_boot_northern_aus, bootstrap_dir, 'giss_22_boot_northern_aus')
save_netcdf_compression(giss_23_boot_northern_aus, bootstrap_dir, 'giss_23_boot_northern_aus')
save_netcdf_compression(giss_24_boot_northern_aus, bootstrap_dir, 'giss_24_boot_northern_aus')
save_netcdf_compression(giss_25_boot_northern_aus, bootstrap_dir, 'giss_25_boot_northern_aus')
save_netcdf_compression(giss_26_boot_northern_aus, bootstrap_dir, 'giss_26_boot_northern_aus')
save_netcdf_compression(giss_27_boot_northern_aus, bootstrap_dir, 'giss_27_boot_northern_aus')
save_netcdf_compression(giss_28_boot_northern_aus, bootstrap_dir, 'giss_28_boot_northern_aus')
save_netcdf_compression(hadcm3_boot_northern_aus, bootstrap_dir, 'hadcm3_boot_northern_aus')
save_netcdf_compression(ipsl_boot_northern_aus, bootstrap_dir, 'ipsl_boot_northern_aus')
save_netcdf_compression(miroc_boot_northern_aus, bootstrap_dir, 'miroc_boot_northern_aus')
save_netcdf_compression(mpi_boot_northern_aus, bootstrap_dir, 'mpi_boot_northern_aus')
save_netcdf_compression(mri_boot_northern_aus, bootstrap_dir, 'mri_boot_northern_aus')
save_netcdf_compression(ff1_boot_northern_aus, bootstrap_dir, 'ff1_boot_northern_aus')
save_netcdf_compression(ff2_boot_northern_aus, bootstrap_dir, 'ff2_boot_northern_aus')
save_netcdf_compression(ff3_boot_northern_aus, bootstrap_dir, 'ff3_boot_northern_aus')
save_netcdf_compression(ff4_boot_northern_aus, bootstrap_dir, 'ff4_boot_northern_aus')
save_netcdf_compression(ff5_boot_northern_aus, bootstrap_dir, 'ff5_boot_northern_aus')
save_netcdf_compression(ff6_boot_northern_aus, bootstrap_dir, 'ff6_boot_northern_aus')
save_netcdf_compression(ff7_boot_northern_aus, bootstrap_dir, 'ff7_boot_northern_aus')
save_netcdf_compression(ff8_boot_northern_aus, bootstrap_dir, 'ff8_boot_northern_aus')
save_netcdf_compression(ff9_boot_northern_aus, bootstrap_dir, 'ff9_boot_northern_aus')
save_netcdf_compression(ff10_boot_northern_aus, bootstrap_dir, 'ff10_boot_northern_aus')
save_netcdf_compression(ff11_boot_northern_aus, bootstrap_dir, 'ff11_boot_northern_aus')
save_netcdf_compression(ff12_boot_northern_aus, bootstrap_dir, 'ff12_boot_northern_aus')
save_netcdf_compression(ff13_boot_northern_aus, bootstrap_dir, 'ff13_boot_northern_aus')
save_netcdf_compression(ff_all_precip_lm_annual_northern_aus_mean_rc_boots,   bootstrap_dir, 'ff_all_boot_northern_aus')
save_netcdf_compression(giss_all_precip_lm_annual_northern_aus_mean_rc_boots, bootstrap_dir, 'giss_all_boot_northern_aus')

# Eastern Aus

In [46]:
def get_eastern_aus_mean(ds):
    # import shapefile using geopandas
    states = gp.read_file('/Users/nickywright/Data/Coastlines/NaturalEarthData/ne_50m_admin_1_states_provinces/ne_50m_admin_1_states_provinces.shp')
    states_sub = states.loc[states['iso_3166_2'].isin(['AU-QLD', 'AU-NSW', 'AU-VIC'])]
    mask = regionmask.mask_geopandas(states_sub, ds.lon, ds.lat)
    # mask out 
    ds_east = ds.where(mask == mask, drop=True)
    ds_eastern_mean = climate_xr_funcs.weighted_mean_CAM(ds_east)
    
    return ds_eastern_mean

In [47]:
ff1_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff1_precip_hist_annual_aus)
ff2_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff2_precip_hist_annual_aus)
ff3_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff3_precip_hist_annual_aus)
ff4_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff4_precip_hist_annual_aus)
ff5_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff5_precip_hist_annual_aus)
ff6_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff6_precip_hist_annual_aus)
ff7_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff7_precip_hist_annual_aus)
ff8_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff8_precip_hist_annual_aus)
ff9_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(ff9_precip_hist_annual_aus)
ff10_precip_hist_annual_eastern_aus       = get_eastern_aus_mean(ff10_precip_hist_annual_aus)
ff11_precip_hist_annual_eastern_aus       = get_eastern_aus_mean(ff11_precip_hist_annual_aus)
ff12_precip_hist_annual_eastern_aus       = get_eastern_aus_mean(ff12_precip_hist_annual_aus)
ff13_precip_hist_annual_eastern_aus       = get_eastern_aus_mean(ff13_precip_hist_annual_aus)

bcc_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(bcc_precip_hist_annual_aus)
ccsm4_precip_hist_annual_eastern_aus      = get_eastern_aus_mean(ccsm4_precip_hist_annual_aus)
csiro_mk3l_precip_hist_annual_eastern_aus = get_eastern_aus_mean(csiro_mk3l_precip_hist_annual_aus)
fgoals_gl_precip_hist_annual_eastern_aus  = get_eastern_aus_mean(fgoals_gl_precip_hist_annual_aus)
fgoals_s2_precip_hist_annual_eastern_aus  = get_eastern_aus_mean(fgoals_s2_precip_hist_annual_aus)
giss_21_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_21_precip_hist_annual_aus)
giss_22_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_22_precip_hist_annual_aus)
giss_23_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_23_precip_hist_annual_aus)
giss_24_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_24_precip_hist_annual_aus)
giss_25_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_25_precip_hist_annual_aus)
giss_26_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_26_precip_hist_annual_aus)
giss_27_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_27_precip_hist_annual_aus)
giss_28_precip_hist_annual_eastern_aus    = get_eastern_aus_mean(giss_28_precip_hist_annual_aus)
hadcm3_precip_hist_annual_eastern_aus     = get_eastern_aus_mean(hadcm3_precip_hist_annual_aus)
ipsl_precip_hist_annual_eastern_aus       = get_eastern_aus_mean(ipsl_precip_hist_annual_aus)
miroc_precip_hist_annual_eastern_aus      = get_eastern_aus_mean(miroc_precip_hist_annual_aus)
mpi_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(mpi_precip_hist_annual_aus)
mri_precip_hist_annual_eastern_aus        = get_eastern_aus_mean(mri_precip_hist_annual_aus)

ff1_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff1_precip_lm_annual_aus)
ff2_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff2_precip_lm_annual_aus)
ff3_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff3_precip_lm_annual_aus)
ff4_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff4_precip_lm_annual_aus)
ff5_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff5_precip_lm_annual_aus)
ff6_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff6_precip_lm_annual_aus)
ff7_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff7_precip_lm_annual_aus)
ff8_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff8_precip_lm_annual_aus)
ff9_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(ff9_precip_lm_annual_aus)
ff10_precip_lm_annual_eastern_aus       = get_eastern_aus_mean(ff10_precip_lm_annual_aus)
ff11_precip_lm_annual_eastern_aus       = get_eastern_aus_mean(ff11_precip_lm_annual_aus)
ff12_precip_lm_annual_eastern_aus       = get_eastern_aus_mean(ff12_precip_lm_annual_aus)
ff13_precip_lm_annual_eastern_aus       = get_eastern_aus_mean(ff13_precip_lm_annual_aus)
bcc_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(bcc_precip_lm_annual_aus)
ccsm4_precip_lm_annual_eastern_aus      = get_eastern_aus_mean(ccsm4_precip_lm_annual_aus)
csiro_mk3l_precip_lm_annual_eastern_aus = get_eastern_aus_mean(csiro_mk3l_precip_lm_annual_aus)
fgoals_gl_precip_lm_annual_eastern_aus  = get_eastern_aus_mean(fgoals_gl_precip_lm_annual_aus)
fgoals_s2_precip_lm_annual_eastern_aus  = get_eastern_aus_mean(fgoals_s2_precip_lm_annual_aus)
giss_21_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_21_precip_lm_annual_aus)
giss_22_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_22_precip_lm_annual_aus)
giss_23_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_23_precip_lm_annual_aus)
giss_24_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_24_precip_lm_annual_aus)
giss_25_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_25_precip_lm_annual_aus)
giss_26_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_26_precip_lm_annual_aus)
giss_27_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_27_precip_lm_annual_aus)
giss_28_precip_lm_annual_eastern_aus    = get_eastern_aus_mean(giss_28_precip_lm_annual_aus)
hadcm3_precip_lm_annual_eastern_aus     = get_eastern_aus_mean(hadcm3_precip_lm_annual_aus)
ipsl_precip_lm_annual_eastern_aus       = get_eastern_aus_mean(ipsl_precip_lm_annual_aus)
miroc_precip_lm_annual_eastern_aus      = get_eastern_aus_mean(miroc_precip_lm_annual_aus)
mpi_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(mpi_precip_lm_annual_aus)
mri_precip_lm_annual_eastern_aus        = get_eastern_aus_mean(mri_precip_lm_annual_aus)

awap_masked_annual_eastern_aus = get_eastern_aus_mean(awap_gf_annual)

In [48]:
ff1_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff1_precip_hist_annual_eastern_aus, 1900, 2000)    
ff2_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff2_precip_hist_annual_eastern_aus, 1900, 2000)    
ff3_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff3_precip_hist_annual_eastern_aus, 1900, 2000)    
ff4_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff4_precip_hist_annual_eastern_aus, 1900, 2000)    
ff5_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff5_precip_hist_annual_eastern_aus, 1900, 2000)    
ff6_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff6_precip_hist_annual_eastern_aus, 1900, 2000)    
ff7_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff7_precip_hist_annual_eastern_aus, 1900, 2000)    
ff8_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff8_precip_hist_annual_eastern_aus, 1900, 2000)    
ff9_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff9_precip_hist_annual_eastern_aus, 1900, 2000)    
ff10_precip_hist_annual_eastern_aus_rc      = droughts_lastmillennium_fromyear(ff10_precip_hist_annual_eastern_aus, 1900, 2000)    
ff11_precip_hist_annual_eastern_aus_rc      = droughts_lastmillennium_fromyear(ff11_precip_hist_annual_eastern_aus, 1900, 2000)    
ff12_precip_hist_annual_eastern_aus_rc      = droughts_lastmillennium_fromyear(ff12_precip_hist_annual_eastern_aus, 1900, 2000)    
ff13_precip_hist_annual_eastern_aus_rc      = droughts_lastmillennium_fromyear(ff13_precip_hist_annual_eastern_aus, 1900, 2000)    
bcc_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(bcc_precip_hist_annual_eastern_aus, 1900, 2000)    
ccsm4_precip_hist_annual_eastern_aus_rc     = droughts_lastmillennium_fromyear(ccsm4_precip_hist_annual_eastern_aus     , 1900, 2000)
csiro_mk3l_precip_hist_annual_eastern_aus_rc= droughts_lastmillennium_fromyear(csiro_mk3l_precip_hist_annual_eastern_aus, 1900, 2000)
fgoals_gl_precip_hist_annual_eastern_aus_rc = droughts_lastmillennium_fromyear(fgoals_gl_precip_hist_annual_eastern_aus , 1900, 2000)
fgoals_s2_precip_hist_annual_eastern_aus_rc = droughts_lastmillennium_fromyear(fgoals_s2_precip_hist_annual_eastern_aus , 1900, 2000)
giss_21_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_21_precip_hist_annual_eastern_aus   , 1900, 2000)
giss_22_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_22_precip_hist_annual_eastern_aus   , 1900, 2000)
giss_23_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_23_precip_hist_annual_eastern_aus   , 1900, 2000)
giss_24_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_24_precip_hist_annual_eastern_aus   , 1900, 2000)
giss_25_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_25_precip_hist_annual_eastern_aus   , 1900, 2000)
giss_26_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_26_precip_hist_annual_eastern_aus   , 1900, 2000)
giss_27_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_27_precip_hist_annual_eastern_aus   , 1900, 2000)
giss_28_precip_hist_annual_eastern_aus_rc   = droughts_lastmillennium_fromyear(giss_28_precip_hist_annual_eastern_aus   , 1900, 2000)
hadcm3_precip_hist_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(hadcm3_precip_hist_annual_eastern_aus    , 1900, 2000)
ipsl_precip_hist_annual_eastern_aus_rc      = droughts_lastmillennium_fromyear(ipsl_precip_hist_annual_eastern_aus      , 1900, 2000)
miroc_precip_hist_annual_eastern_aus_rc     = droughts_lastmillennium_fromyear(miroc_precip_hist_annual_eastern_aus     , 1900, 2000)
mpi_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(mpi_precip_hist_annual_eastern_aus       , 1900, 2000)
mri_precip_hist_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(mri_precip_hist_annual_eastern_aus       , 1900, 2000)

ff1_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff1_precip_lm_annual_eastern_aus       , 1900, 2000)
ff2_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff2_precip_lm_annual_eastern_aus       , 1900, 2000)
ff3_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff3_precip_lm_annual_eastern_aus       , 1900, 2000)
ff4_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff4_precip_lm_annual_eastern_aus       , 1900, 2000)
ff5_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff5_precip_lm_annual_eastern_aus       , 1900, 2000)
ff6_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff6_precip_lm_annual_eastern_aus       , 1900, 2000)
ff7_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff7_precip_lm_annual_eastern_aus       , 1900, 2000)
ff8_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff8_precip_lm_annual_eastern_aus       , 1900, 2000)
ff9_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(ff9_precip_lm_annual_eastern_aus       , 1900, 2000)
ff10_precip_lm_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff10_precip_lm_annual_eastern_aus      , 1900, 2000)
ff11_precip_lm_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff11_precip_lm_annual_eastern_aus      , 1900, 2000)
ff12_precip_lm_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff12_precip_lm_annual_eastern_aus      , 1900, 2000)
ff13_precip_lm_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ff13_precip_lm_annual_eastern_aus      , 1900, 2000)
bcc_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(bcc_precip_lm_annual_eastern_aus       , 1900, 2000)
ccsm4_precip_lm_annual_eastern_aus_rc      = droughts_lastmillennium_fromyear(ccsm4_precip_lm_annual_eastern_aus     , 1900, 2000)
csiro_mk3l_precip_lm_annual_eastern_aus_rc = droughts_lastmillennium_fromyear(csiro_mk3l_precip_lm_annual_eastern_aus, 1900, 2000)
fgoals_gl_precip_lm_annual_eastern_aus_rc  = droughts_lastmillennium_fromyear(fgoals_gl_precip_lm_annual_eastern_aus , 1900, 2000)
fgoals_s2_precip_lm_annual_eastern_aus_rc  = droughts_lastmillennium_fromyear(fgoals_s2_precip_lm_annual_eastern_aus , 1900, 2000)
giss_21_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_21_precip_lm_annual_eastern_aus   , 1900, 2000)
giss_22_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_22_precip_lm_annual_eastern_aus   , 1900, 2000)
giss_23_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_23_precip_lm_annual_eastern_aus   , 1900, 2000)
giss_24_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_24_precip_lm_annual_eastern_aus   , 1900, 2000)
giss_25_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_25_precip_lm_annual_eastern_aus   , 1900, 2000)
giss_26_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_26_precip_lm_annual_eastern_aus   , 1900, 2000)
giss_27_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_27_precip_lm_annual_eastern_aus   , 1900, 2000)
giss_28_precip_lm_annual_eastern_aus_rc    = droughts_lastmillennium_fromyear(giss_28_precip_lm_annual_eastern_aus   , 1900, 2000)
hadcm3_precip_lm_annual_eastern_aus_rc     = droughts_lastmillennium_fromyear(hadcm3_precip_lm_annual_eastern_aus    , 1900, 2000)
ipsl_precip_lm_annual_eastern_aus_rc       = droughts_lastmillennium_fromyear(ipsl_precip_lm_annual_eastern_aus      , 1900, 2000)
miroc_precip_lm_annual_eastern_aus_rc      = droughts_lastmillennium_fromyear(miroc_precip_lm_annual_eastern_aus     , 1900, 2000)
mpi_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(mpi_precip_lm_annual_eastern_aus       , 1900, 2000)
mri_precip_lm_annual_eastern_aus_rc        = droughts_lastmillennium_fromyear(mri_precip_lm_annual_eastern_aus       , 1900, 2000)

awap_masked_annual_eastern_aus_rc = droughts_lastmillennium_fromyear(awap_masked_annual_eastern_aus, 1900, 2000)

ff_all_precip_hist_annual_eastern_aus = xr.concat([ff1_precip_hist_annual_eastern_aus, ff2_precip_hist_annual_eastern_aus,
                                                    ff3_precip_hist_annual_eastern_aus, ff4_precip_hist_annual_eastern_aus, 
                                                    ff5_precip_hist_annual_eastern_aus, ff6_precip_hist_annual_eastern_aus, 
                                                    ff7_precip_hist_annual_eastern_aus, ff8_precip_hist_annual_eastern_aus, 
                                                    ff9_precip_hist_annual_eastern_aus, ff10_precip_hist_annual_eastern_aus,
                                                    ff11_precip_hist_annual_eastern_aus, ff12_precip_hist_annual_eastern_aus,
                                                    ff13_precip_hist_annual_eastern_aus], dim='en')
ff_all_precip_hist_annual_eastern_aus_mean = ff_all_precip_hist_annual_eastern_aus.mean(dim='en')
ff_all_precip_hist_annual_eastern_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_hist_annual_eastern_aus_mean, 1900,2000)

giss_all_precip_hist_annual_eastern_aus = xr.concat([giss_21_precip_hist_annual_eastern_aus,
                                                      giss_22_precip_hist_annual_eastern_aus,
                                                      giss_23_precip_hist_annual_eastern_aus,
                                                      giss_24_precip_hist_annual_eastern_aus,
                                                      giss_25_precip_hist_annual_eastern_aus,
                                                      giss_26_precip_hist_annual_eastern_aus,
                                                      giss_27_precip_hist_annual_eastern_aus,
                                                      giss_28_precip_hist_annual_eastern_aus], dim='en')
giss_all_precip_hist_annual_eastern_aus_mean = giss_all_precip_hist_annual_eastern_aus.mean(dim='en')
giss_all_precip_hist_annual_eastern_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_hist_annual_eastern_aus_mean, 1900, 2000)

ff_all_precip_lm_annual_eastern_aus = xr.concat([ff1_precip_lm_annual_eastern_aus, ff2_precip_lm_annual_eastern_aus,
                                                  ff3_precip_lm_annual_eastern_aus, ff4_precip_lm_annual_eastern_aus, 
                                                  ff5_precip_lm_annual_eastern_aus, ff6_precip_lm_annual_eastern_aus, 
                                                  ff7_precip_lm_annual_eastern_aus, ff8_precip_lm_annual_eastern_aus, 
                                                  ff9_precip_lm_annual_eastern_aus, ff10_precip_lm_annual_eastern_aus,
                                                  ff11_precip_lm_annual_eastern_aus, ff12_precip_lm_annual_eastern_aus,
                                                  ff13_precip_lm_annual_eastern_aus], dim='en')
ff_all_precip_lm_annual_eastern_aus_mean = ff_all_precip_lm_annual_eastern_aus.mean(dim='en')
ff_all_precip_lm_annual_eastern_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_lm_annual_eastern_aus_mean, 1900,2000)

giss_all_precip_lm_annual_eastern_aus = xr.concat([giss_21_precip_lm_annual_eastern_aus,
                                                    giss_22_precip_lm_annual_eastern_aus,
                                                    giss_23_precip_lm_annual_eastern_aus,
                                                    giss_24_precip_lm_annual_eastern_aus,
                                                    giss_25_precip_lm_annual_eastern_aus,
                                                    giss_26_precip_lm_annual_eastern_aus,
                                                    giss_27_precip_lm_annual_eastern_aus,
                                                    giss_28_precip_lm_annual_eastern_aus], dim='en')
giss_all_precip_lm_annual_eastern_aus_mean = giss_all_precip_lm_annual_eastern_aus.mean(dim='en')
giss_all_precip_lm_annual_eastern_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_lm_annual_eastern_aus_mean, 1900, 2000)

In [49]:
save_netcdf_compression(ff1_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff1_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff2_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff2_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff3_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff3_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff4_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff4_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff5_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff5_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff6_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff6_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff7_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff7_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff8_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff8_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff9_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff9_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff10_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff10_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff11_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff11_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff12_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff12_precip_hist_annual_eastern_aus')
save_netcdf_compression(ff13_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ff13_precip_hist_annual_eastern_aus')

save_netcdf_compression(bcc_precip_hist_annual_eastern_aus_rc,   input_dir_hist, 'bcc_precip_hist_annual_eastern_aus')
save_netcdf_compression(ccsm4_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'ccsm4_precip_hist_annual_eastern_aus')
save_netcdf_compression(csiro_mk3l_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'csiro_mk3l_precip_hist_annual_eastern_aus')
save_netcdf_compression(fgoals_gl_precip_hist_annual_eastern_aus_rc,  input_dir_hist, 'fgoals_gl_precip_hist_annual_eastern_aus')
save_netcdf_compression(fgoals_s2_precip_hist_annual_eastern_aus_rc,  input_dir_hist, 'fgoals_s2_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_21_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_21_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_22_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_22_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_23_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_23_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_24_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_24_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_25_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_25_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_26_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_26_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_27_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_27_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_28_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'giss_28_precip_hist_annual_eastern_aus')
save_netcdf_compression(hadcm3_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'hadcm3_precip_hist_annual_eastern_aus')
save_netcdf_compression(ipsl_precip_hist_annual_eastern_aus_rc,   input_dir_hist, 'ipsl_precip_hist_annual_eastern_aus')
save_netcdf_compression(miroc_precip_hist_annual_eastern_aus_rc, input_dir_hist, 'miroc_precip_hist_annual_eastern_aus')
save_netcdf_compression(mpi_precip_hist_annual_eastern_aus_rc,   input_dir_hist, 'mpi_precip_hist_annual_eastern_aus')
save_netcdf_compression(mri_precip_hist_annual_eastern_aus_rc,   input_dir_hist, 'mri_precip_hist_annual_eastern_aus')

save_netcdf_compression(ff_all_precip_hist_annual_eastern_aus_mean_rc,   input_dir_hist, 'ff_all_precip_hist_annual_eastern_aus')
save_netcdf_compression(giss_all_precip_hist_annual_eastern_aus_mean_rc, input_dir_hist, 'giss_all_precip_hist_annual_eastern_aus')

save_netcdf_compression(awap_masked_annual_eastern_aus_rc, input_dir_hist, 'awap_masked_annual_eastern_aus')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [50]:
save_netcdf_compression(ff1_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff1_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff2_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff2_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff3_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff3_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff4_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff4_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff5_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff5_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff6_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff6_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff7_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff7_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff8_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff8_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff9_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff9_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff10_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff10_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff11_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff11_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff12_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff12_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff13_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ff13_precip_lm_annual_eastern_aus')

save_netcdf_compression(bcc_precip_lm_annual_eastern_aus_rc,   input_dir_lm, 'bcc_precip_lm_annual_eastern_aus')
save_netcdf_compression(ccsm4_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'ccsm4_precip_lm_annual_eastern_aus')
save_netcdf_compression(csiro_mk3l_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'csiro_mk3l_precip_lm_annual_eastern_aus')
save_netcdf_compression(fgoals_gl_precip_lm_annual_eastern_aus_rc,  input_dir_lm, 'fgoals_gl_precip_lm_annual_eastern_aus')
save_netcdf_compression(fgoals_s2_precip_lm_annual_eastern_aus_rc,  input_dir_lm, 'fgoals_s2_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_21_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_21_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_22_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_22_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_23_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_23_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_24_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_24_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_25_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_25_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_26_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_26_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_27_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_27_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_28_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'giss_28_precip_lm_annual_eastern_aus')
save_netcdf_compression(hadcm3_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'hadcm3_precip_lm_annual_eastern_aus')
save_netcdf_compression(ipsl_precip_lm_annual_eastern_aus_rc,   input_dir_lm, 'ipsl_precip_lm_annual_eastern_aus')
save_netcdf_compression(miroc_precip_lm_annual_eastern_aus_rc,  input_dir_lm, 'miroc_precip_lm_annual_eastern_aus')
save_netcdf_compression(mpi_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'mpi_precip_lm_annual_eastern_aus')
save_netcdf_compression(mri_precip_lm_annual_eastern_aus_rc, input_dir_lm, 'mri_precip_lm_annual_eastern_aus')
save_netcdf_compression(giss_all_precip_lm_annual_eastern_aus_mean_rc, input_dir_lm, 'giss_all_precip_lm_annual_eastern_aus')
save_netcdf_compression(ff_all_precip_lm_annual_eastern_aus_mean_rc,   input_dir_lm, 'ff_all_precip_lm_annual_eastern_aus')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

ds

In [51]:
ds_eastern_aus_prect = awap_masked_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect = ds_eastern_aus_prect.to_dataset()
ds_eastern_aus_prect['AWAP'] = ds_eastern_aus_prect.PRECT_mm
ds_eastern_aus_prect = ds_eastern_aus_prect.drop(['PRECT_mm'])

ds_eastern_aus_prect['bcc'] = bcc_precip_hist_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect['ccsm4'] = ccsm4_precip_hist_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect['csiro_mk3l'] = csiro_mk3l_precip_hist_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect['fgoals_gl'] = fgoals_gl_precip_hist_annual_eastern_aus_rc.PRECT_mm 
ds_eastern_aus_prect['fgoals_s2'] = fgoals_s2_precip_hist_annual_eastern_aus_rc.PRECT_mm 
ds_eastern_aus_prect['giss_21'] = giss_21_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['giss_22'] = giss_22_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['giss_23'] = giss_23_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['giss_24'] = giss_24_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['giss_25'] = giss_25_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['giss_26'] = giss_26_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['giss_27'] = giss_27_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['giss_28'] = giss_28_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['hadcm3'] = hadcm3_precip_hist_annual_eastern_aus_rc.PRECT_mm    
ds_eastern_aus_prect['ipsl'] = ipsl_precip_hist_annual_eastern_aus_rc.PRECT_mm      
ds_eastern_aus_prect['miroc'] = miroc_precip_hist_annual_eastern_aus_rc.PRECT_mm     
ds_eastern_aus_prect['mpi'] = mpi_precip_hist_annual_eastern_aus_rc.PRECT_mm       
ds_eastern_aus_prect['mri'] = mri_precip_hist_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect['ff1'] = ff1_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff2'] = ff2_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff3'] = ff3_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff4'] = ff4_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff5'] = ff5_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff6'] = ff6_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff7'] = ff7_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff8'] = ff8_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff9'] = ff9_precip_hist_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect['ff10'] = ff10_precip_hist_annual_eastern_aus_rc.PRECT_mm  
ds_eastern_aus_prect['ff11'] = ff11_precip_hist_annual_eastern_aus_rc.PRECT_mm  
ds_eastern_aus_prect['ff12'] = ff12_precip_hist_annual_eastern_aus_rc.PRECT_mm  
ds_eastern_aus_prect['ff13'] = ff13_precip_hist_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect['ff_all'] = ff_all_precip_hist_annual_eastern_aus_mean_rc.PRECT_mm  
ds_eastern_aus_prect['giss_13'] = giss_all_precip_hist_annual_eastern_aus_mean_rc.PRECT_mm
# ds_eastern_aus_prect = ds_eastern_aus_prect.drop('quantile')
save_netcdf_compression(ds_eastern_aus_prect, input_dir_hist, 'pmip3_models_precip_hist_eastern_aus')


[########################################] | 100% Completed |  0.1s


In [52]:
# for the last millennium
ds_eastern_aus_prect_lm = bcc_precip_lm_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect_lm = ds_eastern_aus_prect_lm.to_dataset()
ds_eastern_aus_prect_lm['bcc'] = ds_eastern_aus_prect_lm.PRECT_mm

ds_eastern_aus_prect_lm['ccsm4'] = ccsm4_precip_lm_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect_lm['csiro_mk3l'] = csiro_mk3l_precip_lm_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect_lm['fgoals_gl'] = fgoals_gl_precip_lm_annual_eastern_aus_rc.PRECT_mm 
ds_eastern_aus_prect_lm['fgoals_s2'] = fgoals_s2_precip_lm_annual_eastern_aus_rc.PRECT_mm 
ds_eastern_aus_prect_lm['giss_21'] = giss_21_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['giss_22'] = giss_22_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['giss_23'] = giss_23_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['giss_24'] = giss_24_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['giss_25'] = giss_25_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['giss_26'] = giss_26_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['giss_27'] = giss_27_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['giss_28'] = giss_28_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['hadcm3'] = hadcm3_precip_lm_annual_eastern_aus_rc.PRECT_mm    
ds_eastern_aus_prect_lm['ipsl'] = ipsl_precip_lm_annual_eastern_aus_rc.PRECT_mm      
ds_eastern_aus_prect_lm['miroc'] = miroc_precip_lm_annual_eastern_aus_rc.PRECT_mm     
ds_eastern_aus_prect_lm['mpi'] = mpi_precip_lm_annual_eastern_aus_rc.PRECT_mm       
ds_eastern_aus_prect_lm['mri'] = mri_precip_lm_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect_lm['ff1'] = ff1_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff2'] = ff2_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff3'] = ff3_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff4'] = ff4_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff5'] = ff5_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff6'] = ff6_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff7'] = ff7_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff8'] = ff8_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff9'] = ff9_precip_lm_annual_eastern_aus_rc.PRECT_mm   
ds_eastern_aus_prect_lm['ff10'] = ff10_precip_lm_annual_eastern_aus_rc.PRECT_mm  
ds_eastern_aus_prect_lm['ff11'] = ff11_precip_lm_annual_eastern_aus_rc.PRECT_mm  
ds_eastern_aus_prect_lm['ff12'] = ff12_precip_lm_annual_eastern_aus_rc.PRECT_mm  
ds_eastern_aus_prect_lm['ff13'] = ff13_precip_lm_annual_eastern_aus_rc.PRECT_mm
ds_eastern_aus_prect_lm['ff_all'] = ff_all_precip_lm_annual_eastern_aus_mean_rc.PRECT_mm
ds_eastern_aus_prect_lm['giss_all'] = giss_all_precip_lm_annual_eastern_aus_mean_rc.PRECT_mm   

ds_eastern_aus_prect_lm = ds_eastern_aus_prect_lm.drop(('PRECT_mm'))
save_netcdf_compression(ds_eastern_aus_prect_lm, input_dir_lm, 'pmip3_models_precip_lm_eastern_aus')


[########################################] | 100% Completed |  0.1s


bootstrap

In [ ]:
bcc_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.bcc, 'bcc')
save_netcdf_compression(bcc_boot_eastern_aus, bootstrap_dir, 'bcc_boot_eastern_aus')

ccsm4_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ccsm4, 'ccsm4')
save_netcdf_compression(ccsm4_boot_eastern_aus, bootstrap_dir, 'ccsm4_boot_eastern_aus')

csiro_mk3l_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.csiro_mk3l, 'csiro_mk3l')
save_netcdf_compression(csiro_mk3l_boot_eastern_aus, bootstrap_dir, 'csiro_mk3l_boot_eastern_aus')

fgoals_gl_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.fgoals_gl, 'fgoals_gl')
save_netcdf_compression(fgoals_gl_boot_eastern_aus, bootstrap_dir, 'fgoals_gl_boot_eastern_aus')

fgoals_s2_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.fgoals_s2, 'fgoals_s2')
save_netcdf_compression(fgoals_s2_boot_eastern_aus, bootstrap_dir, 'fgoals_s2_boot_eastern_aus')

giss_21_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_21, 'giss_21')
save_netcdf_compression(giss_21_boot_eastern_aus, bootstrap_dir, 'giss_21_boot_eastern_aus')

giss_22_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_22, 'giss_22')
save_netcdf_compression(giss_22_boot_eastern_aus, bootstrap_dir, 'giss_22_boot_eastern_aus')

giss_23_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_23, 'giss_23')
save_netcdf_compression(giss_23_boot_eastern_aus, bootstrap_dir, 'giss_23_boot_eastern_aus')

giss_24_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_24, 'giss_24')
save_netcdf_compression(giss_24_boot_eastern_aus, bootstrap_dir, 'giss_24_boot_eastern_aus')

giss_25_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_25, 'giss_25')
save_netcdf_compression(giss_25_boot_eastern_aus, bootstrap_dir, 'giss_25_boot_eastern_aus')

giss_26_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_26, 'giss_26')
save_netcdf_compression(giss_26_boot_eastern_aus, bootstrap_dir, 'giss_26_boot_eastern_aus')

giss_27_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_27, 'giss_27')
save_netcdf_compression(giss_27_boot_eastern_aus, bootstrap_dir, 'giss_27_boot_eastern_aus')

giss_28_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.giss_28, 'giss_28')
save_netcdf_compression(giss_28_boot_eastern_aus, bootstrap_dir, 'giss_28_boot_eastern_aus')

hadcm3_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.hadcm3, 'hadcm3')
save_netcdf_compression(hadcm3_boot_eastern_aus, bootstrap_dir, 'hadcm3_boot_eastern_aus')

ipsl_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ipsl, 'ipsl')
save_netcdf_compression(ipsl_boot_eastern_aus, bootstrap_dir, 'ipsl_boot_eastern_aus')

miroc_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.miroc, 'miroc')
save_netcdf_compression(miroc_boot_eastern_aus, bootstrap_dir, 'miroc_boot_eastern_aus')

mpi_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.mpi, 'mpi')
save_netcdf_compression(mpi_boot_eastern_aus, bootstrap_dir, 'mpi_boot_eastern_aus')

mri_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.mri, 'mri')
save_netcdf_compression(mri_boot_eastern_aus, bootstrap_dir, 'mri_boot_eastern_aus')

ff1_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff1, 'ff1')
save_netcdf_compression(ff1_boot_eastern_aus, bootstrap_dir, 'ff1_boot_eastern_aus')

ff2_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff2, 'ff2')
save_netcdf_compression(ff2_boot_eastern_aus, bootstrap_dir, 'ff2_boot_eastern_aus')

ff3_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff3, 'ff3')
save_netcdf_compression(ff3_boot_eastern_aus, bootstrap_dir, 'ff3_boot_eastern_aus')

ff4_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff4, 'ff4')
save_netcdf_compression(ff4_boot_eastern_aus, bootstrap_dir, 'ff4_boot_eastern_aus')

ff5_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff5, 'ff5')
save_netcdf_compression(ff5_boot_eastern_aus, bootstrap_dir, 'ff5_boot_eastern_aus')

ff6_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff6, 'ff6')
save_netcdf_compression(ff6_boot_eastern_aus, bootstrap_dir 'ff6_boot_eastern_aus')

ff7_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff7, 'ff7')
save_netcdf_compression(ff7_boot_eastern_aus, bootstrap_dir, 'ff7_boot_eastern_aus')

ff8_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff8, 'ff8')
save_netcdf_compression(ff8_boot_eastern_aus, bootstrap_dir, 'ff8_boot_eastern_aus')

ff9_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff9, 'ff9')
save_netcdf_compression(ff9_boot_eastern_aus, bootstrap_dir, 'ff9_boot_eastern_aus')

ff10_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff10, 'ff10')
save_netcdf_compression(ff10_boot_eastern_aus, bootstrap_dir, 'ff10_boot_eastern_aus')

ff11_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff11, 'ff11')
save_netcdf_compression(ff11_boot_eastern_aus, bootstrap_dir, 'ff11_boot_eastern_aus')

ff12_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff12, 'ff12')
save_netcdf_compression(ff12_boot_eastern_aus, bootstrap_dir, 'ff12_boot_eastern_aus')

ff13_boot_eastern_aus = bootstrap_years_101(ds_eastern_aus_prect_lm.ff13, 'ff13')
save_netcdf_compression(ff13_boot_eastern_aus, bootstrap_dir, 'ff13_boot_eastern_aus')

ff_all_precip_lm_annual_eastern_aus_mean_rc_boots = bootstrap_years_101(ff_all_precip_lm_annual_eastern_aus_mean_rc.PRECT_mm, 'PRECT_mm')
save_netcdf_compression(ff_all_precip_lm_annual_eastern_aus_mean_rc_boots, bootstrap_dir, 'ff_all_boot_eastern_aus')

giss_all_precip_lm_annual_eastern_aus_mean_rc_boots = bootstrap_years_101(giss_all_precip_lm_annual_eastern_aus_mean_rc.PRECT_mm, 'PRECT_mm')
save_netcdf_compression(giss_all_precip_lm_annual_eastern_aus_mean_rc_boots, bootstrap_dir, 'giss_all_boot_eastern_aus')

## south-eastern Aus

In [15]:
def get_south_eastern_aus_mean(ds):
    ds_southern = ds.where(ds.lat <= -33, drop=True)
    ds_south_eastern = ds_southern.where(ds_southern.lon >= 135, drop=True)
    ds_south_eastern_mean = climate_xr_funcs.weighted_mean_CAM(ds_south_eastern)    
    return ds_south_eastern_mean

In [17]:
ff1_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff1_precip_hist_annual_aus)
ff2_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff2_precip_hist_annual_aus)
ff3_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff3_precip_hist_annual_aus)
ff4_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff4_precip_hist_annual_aus)
ff5_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff5_precip_hist_annual_aus)
ff6_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff6_precip_hist_annual_aus)
ff7_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff7_precip_hist_annual_aus)
ff8_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff8_precip_hist_annual_aus)
ff9_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ff9_precip_hist_annual_aus)
ff10_precip_hist_annual_se_aus      = get_south_eastern_aus_mean(ff10_precip_hist_annual_aus)
ff11_precip_hist_annual_se_aus      = get_south_eastern_aus_mean(ff11_precip_hist_annual_aus)
ff12_precip_hist_annual_se_aus      = get_south_eastern_aus_mean(ff12_precip_hist_annual_aus)
ff13_precip_hist_annual_se_aus      = get_south_eastern_aus_mean(ff13_precip_hist_annual_aus)

bcc_precip_hist_annual_se_aus        = get_south_eastern_aus_mean(bcc_precip_hist_annual_aus)
ccsm4_precip_hist_annual_se_aus      = get_south_eastern_aus_mean(ccsm4_precip_hist_annual_aus)
csiro_mk3l_precip_hist_annual_se_aus = get_south_eastern_aus_mean(csiro_mk3l_precip_hist_annual_aus)
fgoals_gl_precip_hist_annual_se_aus  = get_south_eastern_aus_mean(fgoals_gl_precip_hist_annual_aus)
fgoals_s2_precip_hist_annual_se_aus  = get_south_eastern_aus_mean(fgoals_s2_precip_hist_annual_aus)
giss_21_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_21_precip_hist_annual_aus)
giss_22_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_22_precip_hist_annual_aus)
giss_23_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_23_precip_hist_annual_aus)
giss_24_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_24_precip_hist_annual_aus)
giss_25_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_25_precip_hist_annual_aus)
giss_26_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_26_precip_hist_annual_aus)
giss_27_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_27_precip_hist_annual_aus)
giss_28_precip_hist_annual_se_aus    = get_south_eastern_aus_mean(giss_28_precip_hist_annual_aus)
hadcm3_precip_hist_annual_se_aus     = get_south_eastern_aus_mean(hadcm3_precip_hist_annual_aus)
ipsl_precip_hist_annual_se_aus       = get_south_eastern_aus_mean(ipsl_precip_hist_annual_aus)
miroc_precip_hist_annual_se_aus      = get_south_eastern_aus_mean(miroc_precip_hist_annual_aus)
mpi_precip_hist_annual_se_aus        = get_south_eastern_aus_mean(mpi_precip_hist_annual_aus)
mri_precip_hist_annual_se_aus        = get_south_eastern_aus_mean(mri_precip_hist_annual_aus)

awap_masked_annual_se_aus = get_south_eastern_aus_mean(awap_gf_annual)

In [18]:
ff1_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff1_precip_lm_annual_aus)
ff2_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff2_precip_lm_annual_aus)
ff3_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff3_precip_lm_annual_aus)
ff4_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff4_precip_lm_annual_aus)
ff5_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff5_precip_lm_annual_aus)
ff6_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff6_precip_lm_annual_aus)
ff7_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff7_precip_lm_annual_aus)
ff8_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff8_precip_lm_annual_aus)
ff9_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(ff9_precip_lm_annual_aus)
ff10_precip_lm_annual_se_aus       = get_south_eastern_aus_mean(ff10_precip_lm_annual_aus)
ff11_precip_lm_annual_se_aus       = get_south_eastern_aus_mean(ff11_precip_lm_annual_aus)
ff12_precip_lm_annual_se_aus       = get_south_eastern_aus_mean(ff12_precip_lm_annual_aus)
ff13_precip_lm_annual_se_aus       = get_south_eastern_aus_mean(ff13_precip_lm_annual_aus)
bcc_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(bcc_precip_lm_annual_aus)
ccsm4_precip_lm_annual_se_aus      = get_south_eastern_aus_mean(ccsm4_precip_lm_annual_aus)
csiro_mk3l_precip_lm_annual_se_aus = get_south_eastern_aus_mean(csiro_mk3l_precip_lm_annual_aus)
fgoals_gl_precip_lm_annual_se_aus  = get_south_eastern_aus_mean(fgoals_gl_precip_lm_annual_aus)
fgoals_s2_precip_lm_annual_se_aus  = get_south_eastern_aus_mean(fgoals_s2_precip_lm_annual_aus)
giss_21_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_21_precip_lm_annual_aus)
giss_22_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_22_precip_lm_annual_aus)
giss_23_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_23_precip_lm_annual_aus)
giss_24_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_24_precip_lm_annual_aus)
giss_25_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_25_precip_lm_annual_aus)
giss_26_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_26_precip_lm_annual_aus)
giss_27_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_27_precip_lm_annual_aus)
giss_28_precip_lm_annual_se_aus    = get_south_eastern_aus_mean(giss_28_precip_lm_annual_aus)
hadcm3_precip_lm_annual_se_aus     = get_south_eastern_aus_mean(hadcm3_precip_lm_annual_aus)
ipsl_precip_lm_annual_se_aus       = get_south_eastern_aus_mean(ipsl_precip_lm_annual_aus)
miroc_precip_lm_annual_se_aus      = get_south_eastern_aus_mean(miroc_precip_lm_annual_aus)
mpi_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(mpi_precip_lm_annual_aus)
mri_precip_lm_annual_se_aus        = get_south_eastern_aus_mean(mri_precip_lm_annual_aus)

In [19]:
ff1_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff1_precip_hist_annual_se_aus, 1900, 2000)    
ff2_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff2_precip_hist_annual_se_aus, 1900, 2000)    
ff3_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff3_precip_hist_annual_se_aus, 1900, 2000)    
ff4_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff4_precip_hist_annual_se_aus, 1900, 2000)    
ff5_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff5_precip_hist_annual_se_aus, 1900, 2000)    
ff6_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff6_precip_hist_annual_se_aus, 1900, 2000)    
ff7_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff7_precip_hist_annual_se_aus, 1900, 2000)    
ff8_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff8_precip_hist_annual_se_aus, 1900, 2000)    
ff9_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff9_precip_hist_annual_se_aus, 1900, 2000)    
ff10_precip_hist_annual_se_aus_rc      = droughts_lastmillennium_fromyear(ff10_precip_hist_annual_se_aus, 1900, 2000)    
ff11_precip_hist_annual_se_aus_rc      = droughts_lastmillennium_fromyear(ff11_precip_hist_annual_se_aus, 1900, 2000)    
ff12_precip_hist_annual_se_aus_rc      = droughts_lastmillennium_fromyear(ff12_precip_hist_annual_se_aus, 1900, 2000)    
ff13_precip_hist_annual_se_aus_rc      = droughts_lastmillennium_fromyear(ff13_precip_hist_annual_se_aus, 1900, 2000)    
bcc_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(bcc_precip_hist_annual_se_aus, 1900, 2000)    
ccsm4_precip_hist_annual_se_aus_rc     = droughts_lastmillennium_fromyear(ccsm4_precip_hist_annual_se_aus     , 1900, 2000)
csiro_mk3l_precip_hist_annual_se_aus_rc= droughts_lastmillennium_fromyear(csiro_mk3l_precip_hist_annual_se_aus, 1900, 2000)
fgoals_gl_precip_hist_annual_se_aus_rc = droughts_lastmillennium_fromyear(fgoals_gl_precip_hist_annual_se_aus , 1900, 2000)
fgoals_s2_precip_hist_annual_se_aus_rc = droughts_lastmillennium_fromyear(fgoals_s2_precip_hist_annual_se_aus , 1900, 2000)
giss_21_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_21_precip_hist_annual_se_aus   , 1900, 2000)
giss_22_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_22_precip_hist_annual_se_aus   , 1900, 2000)
giss_23_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_23_precip_hist_annual_se_aus   , 1900, 2000)
giss_24_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_24_precip_hist_annual_se_aus   , 1900, 2000)
giss_25_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_25_precip_hist_annual_se_aus   , 1900, 2000)
giss_26_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_26_precip_hist_annual_se_aus   , 1900, 2000)
giss_27_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_27_precip_hist_annual_se_aus   , 1900, 2000)
giss_28_precip_hist_annual_se_aus_rc   = droughts_lastmillennium_fromyear(giss_28_precip_hist_annual_se_aus   , 1900, 2000)
hadcm3_precip_hist_annual_se_aus_rc    = droughts_lastmillennium_fromyear(hadcm3_precip_hist_annual_se_aus    , 1900, 2000)
ipsl_precip_hist_annual_se_aus_rc      = droughts_lastmillennium_fromyear(ipsl_precip_hist_annual_se_aus      , 1900, 2000)
miroc_precip_hist_annual_se_aus_rc     = droughts_lastmillennium_fromyear(miroc_precip_hist_annual_se_aus     , 1900, 2000)
mpi_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(mpi_precip_hist_annual_se_aus       , 1900, 2000)
mri_precip_hist_annual_se_aus_rc       = droughts_lastmillennium_fromyear(mri_precip_hist_annual_se_aus       , 1900, 2000)

awap_masked_annual_se_aus_rc = droughts_lastmillennium_fromyear(awap_masked_annual_se_aus, 1900, 2000)

ff_all_precip_hist_annual_se_aus = xr.concat([ff1_precip_hist_annual_se_aus, ff2_precip_hist_annual_se_aus,
                                                    ff3_precip_hist_annual_se_aus, ff4_precip_hist_annual_se_aus, 
                                                    ff5_precip_hist_annual_se_aus, ff6_precip_hist_annual_se_aus, 
                                                    ff7_precip_hist_annual_se_aus, ff8_precip_hist_annual_se_aus, 
                                                    ff9_precip_hist_annual_se_aus, ff10_precip_hist_annual_se_aus,
                                                    ff11_precip_hist_annual_se_aus, ff12_precip_hist_annual_se_aus,
                                                    ff13_precip_hist_annual_se_aus], dim='en')
ff_all_precip_hist_annual_se_aus_mean = ff_all_precip_hist_annual_se_aus.mean(dim='en')
ff_all_precip_hist_annual_se_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_hist_annual_se_aus_mean, 1900,2000)

giss_all_precip_hist_annual_se_aus = xr.concat([giss_21_precip_hist_annual_se_aus,
                                                      giss_22_precip_hist_annual_se_aus,
                                                      giss_23_precip_hist_annual_se_aus,
                                                      giss_24_precip_hist_annual_se_aus,
                                                      giss_25_precip_hist_annual_se_aus,
                                                      giss_26_precip_hist_annual_se_aus,
                                                      giss_27_precip_hist_annual_se_aus,
                                                      giss_28_precip_hist_annual_se_aus], dim='en')
giss_all_precip_hist_annual_se_aus_mean = giss_all_precip_hist_annual_se_aus.mean(dim='en')
giss_all_precip_hist_annual_se_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_hist_annual_se_aus_mean, 1900, 2000)




In [21]:
save_netcdf_compression(ff1_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff1_precip_hist_annual_se_aus')
save_netcdf_compression(ff2_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff2_precip_hist_annual_se_aus')
save_netcdf_compression(ff3_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff3_precip_hist_annual_se_aus')
save_netcdf_compression(ff4_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff4_precip_hist_annual_se_aus')
save_netcdf_compression(ff5_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff5_precip_hist_annual_se_aus')
save_netcdf_compression(ff6_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff6_precip_hist_annual_se_aus')
save_netcdf_compression(ff7_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff7_precip_hist_annual_se_aus')
save_netcdf_compression(ff8_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff8_precip_hist_annual_se_aus')
save_netcdf_compression(ff9_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff9_precip_hist_annual_se_aus')
save_netcdf_compression(ff10_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff10_precip_hist_annual_se_aus')
save_netcdf_compression(ff11_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff11_precip_hist_annual_se_aus')
save_netcdf_compression(ff12_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff12_precip_hist_annual_se_aus')
save_netcdf_compression(ff13_precip_hist_annual_se_aus_rc, input_dir_hist, 'ff13_precip_hist_annual_se_aus')

save_netcdf_compression(bcc_precip_hist_annual_se_aus_rc, input_dir_hist, 'bcc_precip_hist_annual_se_aus')
save_netcdf_compression(ccsm4_precip_hist_annual_se_aus_rc, input_dir_hist, 'ccsm4_precip_hist_annual_se_aus')
save_netcdf_compression(csiro_mk3l_precip_hist_annual_se_aus_rc, input_dir_hist, 'csiro_mk3l_precip_hist_annual_se_aus')
save_netcdf_compression(fgoals_gl_precip_hist_annual_se_aus_rc, input_dir_hist, 'fgoals_gl_precip_hist_annual_se_aus')
save_netcdf_compression(fgoals_s2_precip_hist_annual_se_aus_rc, input_dir_hist, 'fgoals_s2_precip_hist_annual_se_aus')
save_netcdf_compression(giss_21_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_21_precip_hist_annual_se_aus')
save_netcdf_compression(giss_22_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_22_precip_hist_annual_se_aus')
save_netcdf_compression(giss_23_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_23_precip_hist_annual_se_aus')
save_netcdf_compression(giss_24_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_24_precip_hist_annual_se_aus')
save_netcdf_compression(giss_25_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_25_precip_hist_annual_se_aus')
save_netcdf_compression(giss_26_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_26_precip_hist_annual_se_aus')
save_netcdf_compression(giss_27_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_27_precip_hist_annual_se_aus')
save_netcdf_compression(giss_28_precip_hist_annual_se_aus_rc, input_dir_hist, 'giss_28_precip_hist_annual_se_aus')
save_netcdf_compression(hadcm3_precip_hist_annual_se_aus_rc, input_dir_hist, 'hadcm3_precip_hist_annual_se_aus')
save_netcdf_compression(ipsl_precip_hist_annual_se_aus_rc, input_dir_hist, 'ipsl_precip_hist_annual_se_aus')
save_netcdf_compression(miroc_precip_hist_annual_se_aus_rc, input_dir_hist, 'miroc_precip_hist_annual_se_aus')
save_netcdf_compression(mpi_precip_hist_annual_se_aus_rc, input_dir_hist, 'mpi_precip_hist_annual_se_aus')
save_netcdf_compression(mri_precip_hist_annual_se_aus_rc, input_dir_hist, 'mri_precip_hist_annual_se_aus')
save_netcdf_compression(ff_all_precip_hist_annual_se_aus_mean_rc, input_dir_hist, 'ff_all_precip_hist_annual_se_aus')
save_netcdf_compression(giss_all_precip_hist_annual_se_aus_mean_rc, input_dir_hist, 'giss_all_precip_hist_annual_se_aus')


save_netcdf_compression(awap_masked_annual_se_aus_rc, input_dir_hist, 'awap_masked_annual_se_aus')



[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [22]:
ff1_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff1_precip_lm_annual_se_aus       , 1900, 2000)
ff2_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff2_precip_lm_annual_se_aus       , 1900, 2000)
ff3_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff3_precip_lm_annual_se_aus       , 1900, 2000)
ff4_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff4_precip_lm_annual_se_aus       , 1900, 2000)
ff5_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff5_precip_lm_annual_se_aus       , 1900, 2000)
ff6_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff6_precip_lm_annual_se_aus       , 1900, 2000)
ff7_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff7_precip_lm_annual_se_aus       , 1900, 2000)
ff8_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff8_precip_lm_annual_se_aus       , 1900, 2000)
ff9_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(ff9_precip_lm_annual_se_aus       , 1900, 2000)
ff10_precip_lm_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff10_precip_lm_annual_se_aus      , 1900, 2000)
ff11_precip_lm_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff11_precip_lm_annual_se_aus      , 1900, 2000)
ff12_precip_lm_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff12_precip_lm_annual_se_aus      , 1900, 2000)
ff13_precip_lm_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ff13_precip_lm_annual_se_aus      , 1900, 2000)
bcc_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(bcc_precip_lm_annual_se_aus       , 1900, 2000)
ccsm4_precip_lm_annual_se_aus_rc      = droughts_lastmillennium_fromyear(ccsm4_precip_lm_annual_se_aus     , 1900, 2000)
csiro_mk3l_precip_lm_annual_se_aus_rc = droughts_lastmillennium_fromyear(csiro_mk3l_precip_lm_annual_se_aus, 1900, 2000)
fgoals_gl_precip_lm_annual_se_aus_rc  = droughts_lastmillennium_fromyear(fgoals_gl_precip_lm_annual_se_aus , 1900, 2000)
fgoals_s2_precip_lm_annual_se_aus_rc  = droughts_lastmillennium_fromyear(fgoals_s2_precip_lm_annual_se_aus , 1900, 2000)
giss_21_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_21_precip_lm_annual_se_aus   , 1900, 2000)
giss_22_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_22_precip_lm_annual_se_aus   , 1900, 2000)
giss_23_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_23_precip_lm_annual_se_aus   , 1900, 2000)
giss_24_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_24_precip_lm_annual_se_aus   , 1900, 2000)
giss_25_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_25_precip_lm_annual_se_aus   , 1900, 2000)
giss_26_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_26_precip_lm_annual_se_aus   , 1900, 2000)
giss_27_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_27_precip_lm_annual_se_aus   , 1900, 2000)
giss_28_precip_lm_annual_se_aus_rc    = droughts_lastmillennium_fromyear(giss_28_precip_lm_annual_se_aus   , 1900, 2000)
hadcm3_precip_lm_annual_se_aus_rc     = droughts_lastmillennium_fromyear(hadcm3_precip_lm_annual_se_aus    , 1900, 2000)
ipsl_precip_lm_annual_se_aus_rc       = droughts_lastmillennium_fromyear(ipsl_precip_lm_annual_se_aus      , 1900, 2000)
miroc_precip_lm_annual_se_aus_rc      = droughts_lastmillennium_fromyear(miroc_precip_lm_annual_se_aus     , 1900, 2000)
mpi_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(mpi_precip_lm_annual_se_aus       , 1900, 2000)
mri_precip_lm_annual_se_aus_rc        = droughts_lastmillennium_fromyear(mri_precip_lm_annual_se_aus       , 1900, 2000)

In [23]:
ff_all_precip_lm_annual_se_aus = xr.concat([ff1_precip_lm_annual_se_aus, ff2_precip_lm_annual_se_aus,
                                                    ff3_precip_lm_annual_se_aus, ff4_precip_lm_annual_se_aus, 
                                                    ff5_precip_lm_annual_se_aus, ff6_precip_lm_annual_se_aus, 
                                                    ff7_precip_lm_annual_se_aus, ff8_precip_lm_annual_se_aus, 
                                                    ff9_precip_lm_annual_se_aus, ff10_precip_lm_annual_se_aus,
                                                    ff11_precip_lm_annual_se_aus, ff12_precip_lm_annual_se_aus,
                                                    ff13_precip_lm_annual_se_aus], dim='en')
ff_all_precip_lm_annual_se_aus_mean = ff_all_precip_lm_annual_se_aus.mean(dim='en')
ff_all_precip_lm_annual_se_aus_mean_rc  = droughts_lastmillennium_fromyear(ff_all_precip_lm_annual_se_aus_mean, 1900,2000)

giss_all_precip_lm_annual_se_aus = xr.concat([giss_21_precip_lm_annual_se_aus,
                                                      giss_22_precip_lm_annual_se_aus,
                                                      giss_23_precip_lm_annual_se_aus,
                                                      giss_24_precip_lm_annual_se_aus,
                                                      giss_25_precip_lm_annual_se_aus,
                                                      giss_26_precip_lm_annual_se_aus,
                                                      giss_27_precip_lm_annual_se_aus,
                                                      giss_28_precip_lm_annual_se_aus], dim='en')
giss_all_precip_lm_annual_se_aus_mean = giss_all_precip_lm_annual_se_aus.mean(dim='en')
giss_all_precip_lm_annual_se_aus_mean_rc = droughts_lastmillennium_fromyear(giss_all_precip_lm_annual_se_aus_mean, 1900, 2000)

In [24]:
save_netcdf_compression(ff1_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff1_precip_lm_annual_se_aus')
save_netcdf_compression(ff2_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff2_precip_lm_annual_se_aus')
save_netcdf_compression(ff3_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff3_precip_lm_annual_se_aus')
save_netcdf_compression(ff4_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff4_precip_lm_annual_se_aus')
save_netcdf_compression(ff5_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff5_precip_lm_annual_se_aus')
save_netcdf_compression(ff6_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff6_precip_lm_annual_se_aus')
save_netcdf_compression(ff7_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff7_precip_lm_annual_se_aus')
save_netcdf_compression(ff8_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff8_precip_lm_annual_se_aus')
save_netcdf_compression(ff9_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff9_precip_lm_annual_se_aus')
save_netcdf_compression(ff10_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff10_precip_lm_annual_se_aus')
save_netcdf_compression(ff11_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff11_precip_lm_annual_se_aus')
save_netcdf_compression(ff12_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff12_precip_lm_annual_se_aus')
save_netcdf_compression(ff13_precip_lm_annual_se_aus_rc, input_dir_lm, 'ff13_precip_lm_annual_se_aus')

save_netcdf_compression(bcc_precip_lm_annual_se_aus_rc, input_dir_lm, 'bcc_precip_lm_annual_se_aus')
save_netcdf_compression(ccsm4_precip_lm_annual_se_aus_rc, input_dir_lm, 'ccsm4_precip_lm_annual_se_aus')
save_netcdf_compression(csiro_mk3l_precip_lm_annual_se_aus_rc, input_dir_lm, 'csiro_mk3l_precip_lm_annual_se_aus')
save_netcdf_compression(fgoals_gl_precip_lm_annual_se_aus_rc,  input_dir_lm, 'fgoals_gl_precip_lm_annual_se_aus')
save_netcdf_compression(fgoals_s2_precip_lm_annual_se_aus_rc,  input_dir_lm, 'fgoals_s2_precip_lm_annual_se_aus')
save_netcdf_compression(giss_21_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_21_precip_lm_annual_se_aus')
save_netcdf_compression(giss_22_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_22_precip_lm_annual_se_aus')
save_netcdf_compression(giss_23_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_23_precip_lm_annual_se_aus')
save_netcdf_compression(giss_24_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_24_precip_lm_annual_se_aus')
save_netcdf_compression(giss_25_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_25_precip_lm_annual_se_aus')
save_netcdf_compression(giss_26_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_26_precip_lm_annual_se_aus')
save_netcdf_compression(giss_27_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_27_precip_lm_annual_se_aus')
save_netcdf_compression(giss_28_precip_lm_annual_se_aus_rc, input_dir_lm, 'giss_28_precip_lm_annual_se_aus')
save_netcdf_compression(hadcm3_precip_lm_annual_se_aus_rc, input_dir_lm, 'hadcm3_precip_lm_annual_se_aus')
save_netcdf_compression(ipsl_precip_lm_annual_se_aus_rc,   input_dir_lm, 'ipsl_precip_lm_annual_se_aus')
save_netcdf_compression(miroc_precip_lm_annual_se_aus_rc,  input_dir_lm, 'miroc_precip_lm_annual_se_aus')
save_netcdf_compression(mpi_precip_lm_annual_se_aus_rc, input_dir_lm, 'mpi_precip_lm_annual_se_aus')
save_netcdf_compression(mri_precip_lm_annual_se_aus_rc, input_dir_lm, 'mri_precip_lm_annual_se_aus')

save_netcdf_compression(giss_all_precip_lm_annual_se_aus_mean_rc, input_dir_lm, 'giss_all_precip_lm_annual_se_aus_rc')
save_netcdf_compression(ff_all_precip_lm_annual_se_aus_mean_rc,   input_dir_lm, 'ff_all_precip_lm_annual_se_aus_rc')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [25]:
ds_se_aus_prect_lm = bcc_precip_lm_annual_se_aus_rc.PRECT_mm
ds_se_aus_prect_lm = ds_se_aus_prect_lm.to_dataset()
ds_se_aus_prect_lm['bcc'] = ds_se_aus_prect_lm.PRECT_mm

ds_se_aus_prect_lm['ccsm4'] = ccsm4_precip_lm_annual_se_aus_rc.PRECT_mm
ds_se_aus_prect_lm['csiro_mk3l'] = csiro_mk3l_precip_lm_annual_se_aus_rc.PRECT_mm
ds_se_aus_prect_lm['fgoals_gl'] = fgoals_gl_precip_lm_annual_se_aus_rc.PRECT_mm 
ds_se_aus_prect_lm['fgoals_s2'] = fgoals_s2_precip_lm_annual_se_aus_rc.PRECT_mm 
ds_se_aus_prect_lm['giss_21'] = giss_21_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['giss_22'] = giss_22_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['giss_23'] = giss_23_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['giss_24'] = giss_24_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['giss_25'] = giss_25_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['giss_26'] = giss_26_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['giss_27'] = giss_27_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['giss_28'] = giss_28_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['hadcm3'] = hadcm3_precip_lm_annual_se_aus_rc.PRECT_mm    
ds_se_aus_prect_lm['ipsl'] = ipsl_precip_lm_annual_se_aus_rc.PRECT_mm      
ds_se_aus_prect_lm['miroc'] = miroc_precip_lm_annual_se_aus_rc.PRECT_mm     
ds_se_aus_prect_lm['mpi'] = mpi_precip_lm_annual_se_aus_rc.PRECT_mm       
ds_se_aus_prect_lm['mri'] = mri_precip_lm_annual_se_aus_rc.PRECT_mm
ds_se_aus_prect_lm['ff1'] = ff1_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff2'] = ff2_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff3'] = ff3_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff4'] = ff4_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff5'] = ff5_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff6'] = ff6_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff7'] = ff7_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff8'] = ff8_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff9'] = ff9_precip_lm_annual_se_aus_rc.PRECT_mm   
ds_se_aus_prect_lm['ff10'] = ff10_precip_lm_annual_se_aus_rc.PRECT_mm  
ds_se_aus_prect_lm['ff11'] = ff11_precip_lm_annual_se_aus_rc.PRECT_mm  
ds_se_aus_prect_lm['ff12'] = ff12_precip_lm_annual_se_aus_rc.PRECT_mm  
ds_se_aus_prect_lm['ff13'] = ff13_precip_lm_annual_se_aus_rc.PRECT_mm

ds_se_aus_prect_lm = ds_se_aus_prect_lm.drop(('PRECT_mm'))

save_netcdf_compression(ds_se_aus_prect_lm, input_dir_lm, 'pmip3_models_precip_lm_se_aus')

[########################################] | 100% Completed |  0.1s
